# Mult-Dimensional Poverty
## Picking Top Predictors & Building Final Models
This Jupyter Notebook evaluates over 500 impact indicators and tries to isolate the top 19 indicators that are strong predictors of well-being and are as well "controllable". Controllable is defined using intuition and ensuring that the indicators are something that entrepreneurs could potentially impact.

In [2]:
# Doing all major library imports
import matplotlib.pyplot as plt
import scikitplot as skplt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import re

from sklearn import datasets, metrics
from sklearn.linear_model import LinearRegression, LogisticRegression,LogisticRegressionCV 
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
from matplotlib.colors import ListedColormap
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor, GradientBoostingClassifier, GradientBoostingRegressor, BaggingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVR

plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import scikitplot as skplt
from matplotlib.colors import ListedColormap
from sklearn.metrics import classification_report, confusion_matrix

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.options.display.float_format = '{:.4f}'.format



In [3]:
# Importing files that were generated in the earlier step (B - Imputing Missing Values)

witer = pd.read_csv ('../raw_data/iterated.csv') # with iterated / imputed values
woiter = pd.read_csv ('../raw_data/wo_iterated.csv') # without iterated / imputed values
print (witer.shape)
print (woiter.shape)

(4151, 837)
(4151, 837)


In [4]:
# Checking out the data
witer.head(50)

,country_code,country,date,Access to clean fuels and technologies for cooking (% of population),"Adjusted net enrollment rate, primary (% of primary school age children)","Adjusted net enrollment rate, primary, female (% of primary school age children)","Adjusted net enrollment rate, primary, male (% of primary school age children)",Adjusted net savings (% of GNI),"Adolescent birth rate (births per 1,000 women ages 15-19)","Adolescent fertility rate (births per 1,000 women ages 15-19)",Age dependency ratio (% of working-age population),"Age dependency ratio, old (% of working-age population)","Age dependency ratio, young (% of working-age population)",Agricultural irrigated land (% of total agricultural land),Agricultural land (% of land area),Agricultural land (sq. km),Agricultural production (Index Base: 2004-2006 = 100),Agricultural raw materials exports (% of merchandise exports),Agricultural raw materials imports (% of merchandise imports),"Agriculture, forestry, and fishing, value added (% of GDP)","Agriculture, forestry, and fishing, value added (annual % growth)","Agriculture, forestry, and fishing, value added (constant 2010 US$)","Agriculture, forestry, and fishing, value added per worker (constant 2010 US$)","Air transport, freight (million ton-km)","Air transport, passengers carried","Air transport, registered carrier departures worldwide",Alternative and nuclear energy (% of total energy use),Aquaculture production (metric tons),Arable land (% of land area),Arable land (hectares per person),Arable land (hectares),Arms imports (SIPRI trend indicator values),"Automated teller machines (ATMs) (per 100,000 adults)",Average quality imported,Average unit values imported,"Binding coverage, all products (%)","Binding coverage, manufactured products (%)","Binding coverage, primary products (%)",Births attended by skilled health staff (% of total),"Bound rate, simple mean, all products (%)","Bound rate, simple mean, manufactured products (%)","Bound rate, simple mean, primary products (%)",Broad money (% of GDP),Broad money growth (annual %),Broad money to total reserves ratio,"Burden of customs procedure, WEF (1=extremely inefficient to 7=extremely efficient)",Business extent of disclosure index (0=less disclosure to 10=more disclosure),CO2 emissions (kg per 2010 US$ of GDP),CO2 emissions (kg per 2011 PPP $ of GDP),CO2 emissions (kg per PPP $ of GDP),CO2 emissions (kt),CO2 emissions (metric tons per capita),"CO2 emissions from electricity and heat production, total (% of total fuel combustion)",CO2 emissions from gaseous fuel consumption (% of total),CO2 emissions from gaseous fuel consumption (kt),CO2 emissions from liquid fuel consumption (% of total),CO2 emissions from liquid fuel consumption (kt),CO2 emissions from manufacturing industries and construction (% of total fuel combustion),"CO2 emissions from other sectors, excluding residential buildings and commercial and public services (% of total fuel combustion)",CO2 emissions from residential buildings and commercial and public services (% of total fuel combustion),CO2 emissions from solid fuel consumption (% of total),CO2 emissions from solid fuel consumption (kt),CO2 emissions from transport (% of total fuel combustion),CO2 intensity (kg per kg of oil equivalent energy use),Capture fisheries production (metric tons),Carbon dioxide emissions (kg per 2011 PPP $ of GDP),"Carbon dioxide emissions, per capita (tonnes)","Central government debt, total (% of GDP)","Charges for the use of intellectual property, payments (BoP, current US$)","Charges for the use of intellectual property, receipts (BoP, current US$)",Chemicals (% of value added in manufacturing),Children out of school (% of primary school age),"Children out of school, female (% of female primary school age)","Children out of school, male (% of male primary school age)","Children out of school, primary","Children out of school, primary, female","Children out of school, primary, male",Claims on central governme

In [5]:
# Droping columns and rows based on thresholds that maximize the number of values we have avaible.
# Despite imputing values, some countries and indicators just didn't have enough information to make intelligent imputations.
# For this, I ran a gridsearch of sorts where I evaluated which thresholds save the most values
work = witer.dropna(axis=1, thresh=2600).dropna(axis=0, thresh=735).dropna(axis=1)
print (work.shape)

(1343, 513)


In [6]:
# To mitigate risk of imputed values, I need to ensure that my test set is as robust as possible.
# To do that, I ensured that my test set is full of true values.
true_values = woiter[work.columns]
true_values = true_values[true_values.index.isin (work.index)]
print (true_values.shape)
print (true_values.isnull().sum().sum())
true_values.head()

(1343, 513)
117899


,country_code,country,date,"Adolescent birth rate (births per 1,000 women ages 15-19)","Adolescent fertility rate (births per 1,000 women ages 15-19)",Age dependency ratio (% of working-age population),"Age dependency ratio, old (% of working-age population)","Age dependency ratio, young (% of working-age population)",Agricultural land (% of land area),Agricultural land (sq. km),Agricultural production (Index Base: 2004-2006 = 100),Agricultural raw materials exports (% of merchandise exports),Agricultural raw materials imports (% of merchandise imports),"Agriculture, forestry, and fishing, value added (% of GDP)","Agriculture, forestry, and fishing, value added (annual % growth)","Air transport, freight (million ton-km)","Air transport, passengers carried","Air transport, registered carrier departures worldwide",Arable land (% of land area),Arable land (hectares per person),Arable land (hectares),"Automated teller machines (ATMs) (per 100,000 adults)",Broad money (% of GDP),Broad money growth (annual %),CO2 emissions (kg per 2010 US$ of GDP),CO2 emissions (kg per 2011 PPP $ of GDP),CO2 emissions (kg per PPP $ of GDP),CO2 emissions (kt),CO2 emissions (metric tons per capita),"CO2 emissions from electricity and heat production, total (% of total fuel combustion)",CO2 emissions from liquid fuel consumption (% of total),CO2 emissions from liquid fuel consumption (kt),CO2 emissions from manufacturing industries and construction (% of total fuel combustion),CO2 emissions from residential buildings and commercial and public services (% of total fuel combustion),CO2 emissions from transport (% of total fuel combustion),CO2 intensity (kg per kg of oil equivalent energy use),Capture fisheries production (metric tons),Carbon dioxide emissions (kg per 2011 PPP $ of GDP),"Carbon dioxide emissions, per capita (tonnes)",Claims on central government (annual growth as % of broad money),"Claims on central government, etc. (% GDP)",Coefficient of human inequality,Combustible renewables and waste (% of total energy),"Commercial bank branches (per 100,000 adults)",Commercial service exports (current US$),Commercial service imports (current US$),"Communications, computer, etc. (% of service exports, BoP)","Communications, computer, etc. (% of service imports, BoP)","Computer, communications and other services (% of commercial service exports)","Computer, communications and other services (% of commercial service imports)",Concentration index (exports) (value),"Contributing family workers, female (% of female employment) (modeled ILO estimate)","Contributing family workers, male (% of male employment) (modeled ILO estimate)","Contributing family workers, total (% of total employment) (modeled ILO estimate)",Corruption Perception Index,Crop production index (2004-2006 = 100),Current account balance,Current account balance (% of GDP),"Current account balance (BoP, current US$)",Current health expenditure per capita (current US$),"Current health expenditure per capita, PPP (current international $)",Customs & other import duties,Domestic credit to private sector by banks (% of GDP),Domestic general government health expenditure (% of GDP),Domestic general government health expenditure (% of current health expenditure),Domestic general government health expenditure (% of general government expenditure),Domestic general government health expenditure per capita (current US$),"Domestic general government health expenditure per capita, PPP (current international $)",Domestic private health expenditure (% of current health expenditure),Domestic private health expenditure per capita (current US$),"Domestic private health expenditure per capita, PPP (current international $)",Education index,Electric power consumption (kWh per capita),Electric power transmission and distribution losses (% of output),Electricity production from oil sources (% of total),"Electricity production from oil, gas and coal sources (% of total)",Emissions (thousand metric tons of carbon

In [7]:
# Creating a new column called fill_level to evaluate how many true values each row has
true_values['fill_level'] = true_values.count(axis=1) / len(true_values.columns)

# Isolating the top 200 rows with the most true values / highest fill level
true_values_index = true_values.sort_values (by = 'fill_level', ascending = False).head(200).index
true_values = work[work.index.isin (true_values_index)]
print (true_values.shape)
true_values.head()

(200, 513)


,country_code,country,date,"Adolescent birth rate (births per 1,000 women ages 15-19)","Adolescent fertility rate (births per 1,000 women ages 15-19)",Age dependency ratio (% of working-age population),"Age dependency ratio, old (% of working-age population)","Age dependency ratio, young (% of working-age population)",Agricultural land (% of land area),Agricultural land (sq. km),Agricultural production (Index Base: 2004-2006 = 100),Agricultural raw materials exports (% of merchandise exports),Agricultural raw materials imports (% of merchandise imports),"Agriculture, forestry, and fishing, value added (% of GDP)","Agriculture, forestry, and fishing, value added (annual % growth)","Air transport, freight (million ton-km)","Air transport, passengers carried","Air transport, registered carrier departures worldwide",Arable land (% of land area),Arable land (hectares per person),Arable land (hectares),"Automated teller machines (ATMs) (per 100,000 adults)",Broad money (% of GDP),Broad money growth (annual %),CO2 emissions (kg per 2010 US$ of GDP),CO2 emissions (kg per 2011 PPP $ of GDP),CO2 emissions (kg per PPP $ of GDP),CO2 emissions (kt),CO2 emissions (metric tons per capita),"CO2 emissions from electricity and heat production, total (% of total fuel combustion)",CO2 emissions from liquid fuel consumption (% of total),CO2 emissions from liquid fuel consumption (kt),CO2 emissions from manufacturing industries and construction (% of total fuel combustion),CO2 emissions from residential buildings and commercial and public services (% of total fuel combustion),CO2 emissions from transport (% of total fuel combustion),CO2 intensity (kg per kg of oil equivalent energy use),Capture fisheries production (metric tons),Carbon dioxide emissions (kg per 2011 PPP $ of GDP),"Carbon dioxide emissions, per capita (tonnes)",Claims on central government (annual growth as % of broad money),"Claims on central government, etc. (% GDP)",Coefficient of human inequality,Combustible renewables and waste (% of total energy),"Commercial bank branches (per 100,000 adults)",Commercial service exports (current US$),Commercial service imports (current US$),"Communications, computer, etc. (% of service exports, BoP)","Communications, computer, etc. (% of service imports, BoP)","Computer, communications and other services (% of commercial service exports)","Computer, communications and other services (% of commercial service imports)",Concentration index (exports) (value),"Contributing family workers, female (% of female employment) (modeled ILO estimate)","Contributing family workers, male (% of male employment) (modeled ILO estimate)","Contributing family workers, total (% of total employment) (modeled ILO estimate)",Corruption Perception Index,Crop production index (2004-2006 = 100),Current account balance,Current account balance (% of GDP),"Current account balance (BoP, current US$)",Current health expenditure per capita (current US$),"Current health expenditure per capita, PPP (current international $)",Customs & other import duties,Domestic credit to private sector by banks (% of GDP),Domestic general government health expenditure (% of GDP),Domestic general government health expenditure (% of current health expenditure),Domestic general government health expenditure (% of general government expenditure),Domestic general government health expenditure per capita (current US$),"Domestic general government health expenditure per capita, PPP (current international $)",Domestic private health expenditure (% of current health expenditure),Domestic private health expenditure per capita (current US$),"Domestic private health expenditure per capita, PPP (current international $)",Education index,Electric power consumption (kWh per capita),Electric power transmission and distribution losses (% of output),Electricity production from oil sources (% of total),"Electricity production from oil, gas and coal sources (% of total)",Emissions (thousand metric tons of carbon

In [8]:
# Isolating those rows that have more iterated values than true values
iterated_values = work[~work.index.isin (true_values.index)]
print (iterated_values.shape)
iterated_values.head()

(1143, 513)


,country_code,country,date,"Adolescent birth rate (births per 1,000 women ages 15-19)","Adolescent fertility rate (births per 1,000 women ages 15-19)",Age dependency ratio (% of working-age population),"Age dependency ratio, old (% of working-age population)","Age dependency ratio, young (% of working-age population)",Agricultural land (% of land area),Agricultural land (sq. km),Agricultural production (Index Base: 2004-2006 = 100),Agricultural raw materials exports (% of merchandise exports),Agricultural raw materials imports (% of merchandise imports),"Agriculture, forestry, and fishing, value added (% of GDP)","Agriculture, forestry, and fishing, value added (annual % growth)","Air transport, freight (million ton-km)","Air transport, passengers carried","Air transport, registered carrier departures worldwide",Arable land (% of land area),Arable land (hectares per person),Arable land (hectares),"Automated teller machines (ATMs) (per 100,000 adults)",Broad money (% of GDP),Broad money growth (annual %),CO2 emissions (kg per 2010 US$ of GDP),CO2 emissions (kg per 2011 PPP $ of GDP),CO2 emissions (kg per PPP $ of GDP),CO2 emissions (kt),CO2 emissions (metric tons per capita),"CO2 emissions from electricity and heat production, total (% of total fuel combustion)",CO2 emissions from liquid fuel consumption (% of total),CO2 emissions from liquid fuel consumption (kt),CO2 emissions from manufacturing industries and construction (% of total fuel combustion),CO2 emissions from residential buildings and commercial and public services (% of total fuel combustion),CO2 emissions from transport (% of total fuel combustion),CO2 intensity (kg per kg of oil equivalent energy use),Capture fisheries production (metric tons),Carbon dioxide emissions (kg per 2011 PPP $ of GDP),"Carbon dioxide emissions, per capita (tonnes)",Claims on central government (annual growth as % of broad money),"Claims on central government, etc. (% GDP)",Coefficient of human inequality,Combustible renewables and waste (% of total energy),"Commercial bank branches (per 100,000 adults)",Commercial service exports (current US$),Commercial service imports (current US$),"Communications, computer, etc. (% of service exports, BoP)","Communications, computer, etc. (% of service imports, BoP)","Computer, communications and other services (% of commercial service exports)","Computer, communications and other services (% of commercial service imports)",Concentration index (exports) (value),"Contributing family workers, female (% of female employment) (modeled ILO estimate)","Contributing family workers, male (% of male employment) (modeled ILO estimate)","Contributing family workers, total (% of total employment) (modeled ILO estimate)",Corruption Perception Index,Crop production index (2004-2006 = 100),Current account balance,Current account balance (% of GDP),"Current account balance (BoP, current US$)",Current health expenditure per capita (current US$),"Current health expenditure per capita, PPP (current international $)",Customs & other import duties,Domestic credit to private sector by banks (% of GDP),Domestic general government health expenditure (% of GDP),Domestic general government health expenditure (% of current health expenditure),Domestic general government health expenditure (% of general government expenditure),Domestic general government health expenditure per capita (current US$),"Domestic general government health expenditure per capita, PPP (current international $)",Domestic private health expenditure (% of current health expenditure),Domestic private health expenditure per capita (current US$),"Domestic private health expenditure per capita, PPP (current international $)",Education index,Electric power consumption (kWh per capita),Electric power transmission and distribution losses (% of output),Electricity production from oil sources (% of total),"Electricity production from oil, gas and coal sources (% of total)",Emissions (thousand metric tons of carbon

In [9]:
# Seeing what the year breakout is for our true values set
true_values.date.value_counts()

2010    51
2012    37
2011    36
2014    31
2005    27
2013    18
Name: date, dtype: int64

In [10]:
# Creating a dev set which will consist partially of the true value data set and the iterated value data set
dev_true = true_values[true_values.date.isin (['2005','2014'])]
print (dev_true.shape)
dev_true.head()

(58, 513)


,country_code,country,date,"Adolescent birth rate (births per 1,000 women ages 15-19)","Adolescent fertility rate (births per 1,000 women ages 15-19)",Age dependency ratio (% of working-age population),"Age dependency ratio, old (% of working-age population)","Age dependency ratio, young (% of working-age population)",Agricultural land (% of land area),Agricultural land (sq. km),Agricultural production (Index Base: 2004-2006 = 100),Agricultural raw materials exports (% of merchandise exports),Agricultural raw materials imports (% of merchandise imports),"Agriculture, forestry, and fishing, value added (% of GDP)","Agriculture, forestry, and fishing, value added (annual % growth)","Air transport, freight (million ton-km)","Air transport, passengers carried","Air transport, registered carrier departures worldwide",Arable land (% of land area),Arable land (hectares per person),Arable land (hectares),"Automated teller machines (ATMs) (per 100,000 adults)",Broad money (% of GDP),Broad money growth (annual %),CO2 emissions (kg per 2010 US$ of GDP),CO2 emissions (kg per 2011 PPP $ of GDP),CO2 emissions (kg per PPP $ of GDP),CO2 emissions (kt),CO2 emissions (metric tons per capita),"CO2 emissions from electricity and heat production, total (% of total fuel combustion)",CO2 emissions from liquid fuel consumption (% of total),CO2 emissions from liquid fuel consumption (kt),CO2 emissions from manufacturing industries and construction (% of total fuel combustion),CO2 emissions from residential buildings and commercial and public services (% of total fuel combustion),CO2 emissions from transport (% of total fuel combustion),CO2 intensity (kg per kg of oil equivalent energy use),Capture fisheries production (metric tons),Carbon dioxide emissions (kg per 2011 PPP $ of GDP),"Carbon dioxide emissions, per capita (tonnes)",Claims on central government (annual growth as % of broad money),"Claims on central government, etc. (% GDP)",Coefficient of human inequality,Combustible renewables and waste (% of total energy),"Commercial bank branches (per 100,000 adults)",Commercial service exports (current US$),Commercial service imports (current US$),"Communications, computer, etc. (% of service exports, BoP)","Communications, computer, etc. (% of service imports, BoP)","Computer, communications and other services (% of commercial service exports)","Computer, communications and other services (% of commercial service imports)",Concentration index (exports) (value),"Contributing family workers, female (% of female employment) (modeled ILO estimate)","Contributing family workers, male (% of male employment) (modeled ILO estimate)","Contributing family workers, total (% of total employment) (modeled ILO estimate)",Corruption Perception Index,Crop production index (2004-2006 = 100),Current account balance,Current account balance (% of GDP),"Current account balance (BoP, current US$)",Current health expenditure per capita (current US$),"Current health expenditure per capita, PPP (current international $)",Customs & other import duties,Domestic credit to private sector by banks (% of GDP),Domestic general government health expenditure (% of GDP),Domestic general government health expenditure (% of current health expenditure),Domestic general government health expenditure (% of general government expenditure),Domestic general government health expenditure per capita (current US$),"Domestic general government health expenditure per capita, PPP (current international $)",Domestic private health expenditure (% of current health expenditure),Domestic private health expenditure per capita (current US$),"Domestic private health expenditure per capita, PPP (current international $)",Education index,Electric power consumption (kWh per capita),Electric power transmission and distribution losses (% of output),Electricity production from oil sources (% of total),"Electricity production from oil, gas and coal sources (% of total)",Emissions (thousand metric tons of carbon

In [11]:
# Seeing what years are in the iterated value data set
iterated_values.date.value_counts()

2015    51
2016    49
1996    49
2000    49
1991    48
2009    48
2008    48
2001    48
2017    47
1994    47
1995    47
1999    47
2002    47
2003    47
2004    47
2007    47
1990    47
2006    46
1998    46
1997    46
1993    46
1992    46
2013    31
2005    23
2014    20
2011    13
2012    12
2010     1
Name: date, dtype: int64

In [12]:
# Isolating some years to include in the dev set from the iterated values

dev_iterated = iterated_values[iterated_values.date.isin (['2010','2013', '1993', '2004'])]
print (dev_iterated.shape)

(125, 513)


In [13]:
# Combining the dev_true dataset and the dev_iterated dataset to get our final dev dataset
dev = pd.concat([dev_true,dev_iterated])
print (dev.shape)
dev.head()

(183, 513)


,country_code,country,date,"Adolescent birth rate (births per 1,000 women ages 15-19)","Adolescent fertility rate (births per 1,000 women ages 15-19)",Age dependency ratio (% of working-age population),"Age dependency ratio, old (% of working-age population)","Age dependency ratio, young (% of working-age population)",Agricultural land (% of land area),Agricultural land (sq. km),Agricultural production (Index Base: 2004-2006 = 100),Agricultural raw materials exports (% of merchandise exports),Agricultural raw materials imports (% of merchandise imports),"Agriculture, forestry, and fishing, value added (% of GDP)","Agriculture, forestry, and fishing, value added (annual % growth)","Air transport, freight (million ton-km)","Air transport, passengers carried","Air transport, registered carrier departures worldwide",Arable land (% of land area),Arable land (hectares per person),Arable land (hectares),"Automated teller machines (ATMs) (per 100,000 adults)",Broad money (% of GDP),Broad money growth (annual %),CO2 emissions (kg per 2010 US$ of GDP),CO2 emissions (kg per 2011 PPP $ of GDP),CO2 emissions (kg per PPP $ of GDP),CO2 emissions (kt),CO2 emissions (metric tons per capita),"CO2 emissions from electricity and heat production, total (% of total fuel combustion)",CO2 emissions from liquid fuel consumption (% of total),CO2 emissions from liquid fuel consumption (kt),CO2 emissions from manufacturing industries and construction (% of total fuel combustion),CO2 emissions from residential buildings and commercial and public services (% of total fuel combustion),CO2 emissions from transport (% of total fuel combustion),CO2 intensity (kg per kg of oil equivalent energy use),Capture fisheries production (metric tons),Carbon dioxide emissions (kg per 2011 PPP $ of GDP),"Carbon dioxide emissions, per capita (tonnes)",Claims on central government (annual growth as % of broad money),"Claims on central government, etc. (% GDP)",Coefficient of human inequality,Combustible renewables and waste (% of total energy),"Commercial bank branches (per 100,000 adults)",Commercial service exports (current US$),Commercial service imports (current US$),"Communications, computer, etc. (% of service exports, BoP)","Communications, computer, etc. (% of service imports, BoP)","Computer, communications and other services (% of commercial service exports)","Computer, communications and other services (% of commercial service imports)",Concentration index (exports) (value),"Contributing family workers, female (% of female employment) (modeled ILO estimate)","Contributing family workers, male (% of male employment) (modeled ILO estimate)","Contributing family workers, total (% of total employment) (modeled ILO estimate)",Corruption Perception Index,Crop production index (2004-2006 = 100),Current account balance,Current account balance (% of GDP),"Current account balance (BoP, current US$)",Current health expenditure per capita (current US$),"Current health expenditure per capita, PPP (current international $)",Customs & other import duties,Domestic credit to private sector by banks (% of GDP),Domestic general government health expenditure (% of GDP),Domestic general government health expenditure (% of current health expenditure),Domestic general government health expenditure (% of general government expenditure),Domestic general government health expenditure per capita (current US$),"Domestic general government health expenditure per capita, PPP (current international $)",Domestic private health expenditure (% of current health expenditure),Domestic private health expenditure per capita (current US$),"Domestic private health expenditure per capita, PPP (current international $)",Education index,Electric power consumption (kWh per capita),Electric power transmission and distribution losses (% of output),Electricity production from oil sources (% of total),"Electricity production from oil, gas and coal sources (% of total)",Emissions (thousand metric tons of carbon

In [14]:
# Creating the test set that consists largely of true values
test = true_values[~true_values.index.isin (dev_true.index)]
print (test.shape)
test.head()

(142, 513)


,country_code,country,date,"Adolescent birth rate (births per 1,000 women ages 15-19)","Adolescent fertility rate (births per 1,000 women ages 15-19)",Age dependency ratio (% of working-age population),"Age dependency ratio, old (% of working-age population)","Age dependency ratio, young (% of working-age population)",Agricultural land (% of land area),Agricultural land (sq. km),Agricultural production (Index Base: 2004-2006 = 100),Agricultural raw materials exports (% of merchandise exports),Agricultural raw materials imports (% of merchandise imports),"Agriculture, forestry, and fishing, value added (% of GDP)","Agriculture, forestry, and fishing, value added (annual % growth)","Air transport, freight (million ton-km)","Air transport, passengers carried","Air transport, registered carrier departures worldwide",Arable land (% of land area),Arable land (hectares per person),Arable land (hectares),"Automated teller machines (ATMs) (per 100,000 adults)",Broad money (% of GDP),Broad money growth (annual %),CO2 emissions (kg per 2010 US$ of GDP),CO2 emissions (kg per 2011 PPP $ of GDP),CO2 emissions (kg per PPP $ of GDP),CO2 emissions (kt),CO2 emissions (metric tons per capita),"CO2 emissions from electricity and heat production, total (% of total fuel combustion)",CO2 emissions from liquid fuel consumption (% of total),CO2 emissions from liquid fuel consumption (kt),CO2 emissions from manufacturing industries and construction (% of total fuel combustion),CO2 emissions from residential buildings and commercial and public services (% of total fuel combustion),CO2 emissions from transport (% of total fuel combustion),CO2 intensity (kg per kg of oil equivalent energy use),Capture fisheries production (metric tons),Carbon dioxide emissions (kg per 2011 PPP $ of GDP),"Carbon dioxide emissions, per capita (tonnes)",Claims on central government (annual growth as % of broad money),"Claims on central government, etc. (% GDP)",Coefficient of human inequality,Combustible renewables and waste (% of total energy),"Commercial bank branches (per 100,000 adults)",Commercial service exports (current US$),Commercial service imports (current US$),"Communications, computer, etc. (% of service exports, BoP)","Communications, computer, etc. (% of service imports, BoP)","Computer, communications and other services (% of commercial service exports)","Computer, communications and other services (% of commercial service imports)",Concentration index (exports) (value),"Contributing family workers, female (% of female employment) (modeled ILO estimate)","Contributing family workers, male (% of male employment) (modeled ILO estimate)","Contributing family workers, total (% of total employment) (modeled ILO estimate)",Corruption Perception Index,Crop production index (2004-2006 = 100),Current account balance,Current account balance (% of GDP),"Current account balance (BoP, current US$)",Current health expenditure per capita (current US$),"Current health expenditure per capita, PPP (current international $)",Customs & other import duties,Domestic credit to private sector by banks (% of GDP),Domestic general government health expenditure (% of GDP),Domestic general government health expenditure (% of current health expenditure),Domestic general government health expenditure (% of general government expenditure),Domestic general government health expenditure per capita (current US$),"Domestic general government health expenditure per capita, PPP (current international $)",Domestic private health expenditure (% of current health expenditure),Domestic private health expenditure per capita (current US$),"Domestic private health expenditure per capita, PPP (current international $)",Education index,Electric power consumption (kWh per capita),Electric power transmission and distribution losses (% of output),Electricity production from oil sources (% of total),"Electricity production from oil, gas and coal sources (% of total)",Emissions (thousand metric tons of carbon

In [15]:
# Creating the train set that consists largely of iterated values. This isn't ideal but it is more important for the test set to be real.
train = work[~(work.index.isin (test.index)) & ~(work.index.isin (dev.index))]
print (train.shape)
train.head()

(1018, 513)


,country_code,country,date,"Adolescent birth rate (births per 1,000 women ages 15-19)","Adolescent fertility rate (births per 1,000 women ages 15-19)",Age dependency ratio (% of working-age population),"Age dependency ratio, old (% of working-age population)","Age dependency ratio, young (% of working-age population)",Agricultural land (% of land area),Agricultural land (sq. km),Agricultural production (Index Base: 2004-2006 = 100),Agricultural raw materials exports (% of merchandise exports),Agricultural raw materials imports (% of merchandise imports),"Agriculture, forestry, and fishing, value added (% of GDP)","Agriculture, forestry, and fishing, value added (annual % growth)","Air transport, freight (million ton-km)","Air transport, passengers carried","Air transport, registered carrier departures worldwide",Arable land (% of land area),Arable land (hectares per person),Arable land (hectares),"Automated teller machines (ATMs) (per 100,000 adults)",Broad money (% of GDP),Broad money growth (annual %),CO2 emissions (kg per 2010 US$ of GDP),CO2 emissions (kg per 2011 PPP $ of GDP),CO2 emissions (kg per PPP $ of GDP),CO2 emissions (kt),CO2 emissions (metric tons per capita),"CO2 emissions from electricity and heat production, total (% of total fuel combustion)",CO2 emissions from liquid fuel consumption (% of total),CO2 emissions from liquid fuel consumption (kt),CO2 emissions from manufacturing industries and construction (% of total fuel combustion),CO2 emissions from residential buildings and commercial and public services (% of total fuel combustion),CO2 emissions from transport (% of total fuel combustion),CO2 intensity (kg per kg of oil equivalent energy use),Capture fisheries production (metric tons),Carbon dioxide emissions (kg per 2011 PPP $ of GDP),"Carbon dioxide emissions, per capita (tonnes)",Claims on central government (annual growth as % of broad money),"Claims on central government, etc. (% GDP)",Coefficient of human inequality,Combustible renewables and waste (% of total energy),"Commercial bank branches (per 100,000 adults)",Commercial service exports (current US$),Commercial service imports (current US$),"Communications, computer, etc. (% of service exports, BoP)","Communications, computer, etc. (% of service imports, BoP)","Computer, communications and other services (% of commercial service exports)","Computer, communications and other services (% of commercial service imports)",Concentration index (exports) (value),"Contributing family workers, female (% of female employment) (modeled ILO estimate)","Contributing family workers, male (% of male employment) (modeled ILO estimate)","Contributing family workers, total (% of total employment) (modeled ILO estimate)",Corruption Perception Index,Crop production index (2004-2006 = 100),Current account balance,Current account balance (% of GDP),"Current account balance (BoP, current US$)",Current health expenditure per capita (current US$),"Current health expenditure per capita, PPP (current international $)",Customs & other import duties,Domestic credit to private sector by banks (% of GDP),Domestic general government health expenditure (% of GDP),Domestic general government health expenditure (% of current health expenditure),Domestic general government health expenditure (% of general government expenditure),Domestic general government health expenditure per capita (current US$),"Domestic general government health expenditure per capita, PPP (current international $)",Domestic private health expenditure (% of current health expenditure),Domestic private health expenditure per capita (current US$),"Domestic private health expenditure per capita, PPP (current international $)",Education index,Electric power consumption (kWh per capita),Electric power transmission and distribution losses (% of output),Electricity production from oil sources (% of total),"Electricity production from oil, gas and coal sources (% of total)",Emissions (thousand metric tons of carbon

In [16]:
# Evaluating the shape of all our working sets for Step 1 of this process of isolating top indicators
print (work.shape)
print (train.shape)
print (dev.shape)
print (test.shape)

(1343, 513)
(1018, 513)
(183, 513)
(142, 513)


In [17]:
# Isolating our target indicators

target_indicators = [
       'Mean years of schooling (years)',
       'Gross national income (GNI) per capita (2011 PPP $)',
       'Life expectancy at birth (years)',
       'Expected years of schooling (years)', 
]

target_indicators_all = [
       'Mean years of schooling (years)',
       'Life expectancy index', 
        'Income index',
       'Education index', 
        'Human Development Index (HDI)',
       'Gross national income (GNI) per capita (2011 PPP $)',
       'Life expectancy at birth (years)',
       'Expected years of schooling (years)', 
]

In [18]:
# Predictors to drop because they are too closely related or not controllable.
# This was a manual and iterative process. Every time I ran a model, more of these indicators emerged as strong predictors...
# ...but were not controllable or were too closely related to the component factors of the well-being index (HDI)

predictors_to_drop = [
'GDP per capita, PPP (constant 2011 international $)',
'GDP per capita (2011 PPP $)',
'Estimated gross national income per capita, male (2011 PPP $)',
'GNI per capita, PPP (constant 2011 international $)',
'Estimated gross national income per capita, female (2011 PPP $)',
'GDP per person employed (constant 2011 PPP $)',
'GNI per capita, PPP (current international $)',
'Gross domestic product per capita, constant prices',
'Primary income receipts (BoP, current US$)',
'GDP per capita, PPP (current international $)',
'GNI per capita (constant 2010 US$)',
'GDP per capita (constant 2010 US$)',
'GNI, PPP (current international $)',
'GNI, PPP (constant 2011 international $)',
'GNI, Atlas method (current US$)',
'GNI per capita, Atlas method (current US$)',
'GNI per capita growth (annual %)',
'GNI growth (annual %)',
'GNI (current US$)',
'GNI (constant 2010 US$)',
'GDP, PPP (current international $)',
'GDP, PPP (constant 2011 international $)',
'GDP per unit of energy use (constant 2011 PPP $ per kg of oil equivalent)',
'Gross capital formation (annual % growth)',
'Gross capital formation (constant 2010 US$)',
'Gross capital formation (current US$)',
'Gross domestic product (GDP), total (2011 PPP $ billions)',
'Households and NPISHs Final consumption expenditure per capita (constant 2010 US$)',
'GDP per capita (current US$)',
'Price level ratio of PPP conversion factor (GDP) to market exchange rate',
'Gross savings (% of GDP)',
'Energy use (kg of oil equivalent) per $1,000 GDP (constant 2011 PPP)',
'Energy intensity level of primary energy (MJ/$2011 PPP GDP)',
'Electric power consumption (kWh per capita)',
'CO2 emissions (kg per 2010 US$ of GDP)',
'GDP per unit of energy use (PPP $ per kg of oil equivalent)',
'CO2 emissions (kg per 2011 PPP $ of GDP)',
'CO2 emissions (kg per PPP $ of GDP)',
'Carbon dioxide emissions (kg per 2011 PPP $ of GDP)',
'CO2 intensity (kg per kg of oil equivalent energy use)',
'Out-of-pocket expenditure per capita (current US$)',
'Domestic general government health expenditure (% of current health expenditure)',
'Domestic general government health expenditure per capita (current US$)',
'Carbon dioxide emissions, per capita (tonnes)',
'Mortality rate, under-5 (per 1,000 live births)',
'Mortality rate, female adult (per 1,000 people)',
'Mortality rate, male adult (per 1,000 people)',
'Mortality rate, infant (per 1,000 live births)_y',
'Mortality rate, neonatal (per 1,000 live births)',
'Mortality rate, under-five (per 1,000 live births)',
'Mortality rate, infant (per 1,000 live births)_x',
'Maternal mortality ratio (deaths per 100,000 live births)',
'Population with at least some secondary education (% ages 25 and older)',
'Population with at least some secondary education, female (% ages 25 and older)',
'School enrollment, secondary (% gross)',
'Population with at least some secondary education, male (% ages 25 and older)',
'GDP deflator (base year varies by country)',
'Current health expenditure per capita (current US$)',
'Out-of-pocket expenditure (% of current health expenditure)',
'Current health expenditure per capita, PPP (current international $)',
'Population ages 15-64 (% of total population)',
'Population ages 80 and above, male (% of male population)',
'Population ages 55-59, male (% of male population)',
'Current health expenditure per capita (current US$)',
'GDP deflator: linked series (base year varies by country)',
'PM2.5 air pollution, population exposed to levels exceeding WHO guideline value (% of total)',
'Reserves and related items (BoP, current US$)',
'Inflation, GDP deflator (annual %)',
'Population ages 30-34, female (% of female population)',
'Population ages 55-59, female (% of female population)',
'Population ages 35-39, female (% of female population)',
'Population, male (% of total population)',
'Age dependency ratio, old (% of working-age population)',
'Age dependency ratio, young (% of working-age population)',
'Agriculture, forestry, and fishing, value added (annual % growth)',
'Air transport, freight (million ton-km)',
'Air transport, registered carrier departures worldwide',
'Arable land (hectares)',
'Broad money growth (annual %)',
'CO2 emissions from electricity and heat production, total (% of total fuel combustion)',
'CO2 emissions from liquid fuel consumption (% of total)',
'CO2 emissions from liquid fuel consumption (kt)',
'CO2 emissions from residential buildings and commercial and public services (% of total fuel combustion)',
'CO2 emissions from transport (% of total fuel combustion)',
'Coefficient of human inequality',
'Current account balance',
'Current account balance (BoP, current US$)',
'Food production index (2004-2006 = 100)',
'Foreign direct investment, net inflows (% of GDP)_y',
'Population ages 00-04, female (% of female population)',
'Population ages 00-04, male (% of male population)',
'Population ages 0-14 (% of total population)',
'Population ages 0-14, female',
'Population ages 0-14, female (% of female population)',
'Population ages 0-14, male',
'Population ages 0-14, male (% of male population)',
'Population ages 0-14, total',
'Population ages 05-09, female (% of female population)',
'Population ages 10-14, female (% of female population)',
'Population ages 10-14, male (% of male population)',
'Population ages 15-19, female (% of female population)',
'Population ages 15-64, female',
'Population ages 15-64, male',
'Population ages 15-64, total',
'Population ages 15–64 (millions)',
'Population ages 25-29, female (% of female population)',
'Population ages 25-29, male (% of male population)',
'Population ages 30-34, male (% of male population)',
'Population ages 35-39, male (% of male population)',
'Population ages 40-44, female (% of female population)',
'Population ages 40-44, male (% of male population)',
'Population ages 45-49, female (% of female population)',
'Population ages 45-49, male (% of male population)',
'Population ages 50-54, female (% of female population)',
'Population ages 50-54, male (% of male population)',
'Population ages 60-64, female (% of female population)',
'Population ages 60-64, male (% of male population)',
'Population ages 65 and above (% of total population)',
'Population ages 65 and above, female',
'Population ages 65 and above, female (% of female population)',
'Population ages 65 and above, male',
'Population ages 65 and above, male (% of male population)',
'Population ages 65 and above, total',
'Population ages 65 and older (millions)',
'Population ages 65-69, female (% of female population)',
'Population ages 65-69, male (% of male population)',
'Population ages 70-74, female (% of female population)',
'Population ages 70-74, male (% of male population)',
'Population ages 75-79, female (% of female population)',
'Population ages 75-79, male (% of male population)',
'Population ages 80 and above, female (% of female population)',
'Population density (people per sq. km of land area)',
'Population growth (annual %)',
'Population under age 5 (millions)',
'Population, female',
'Population, male',
'Population, total',
'Renewable energy consumption (% of total final energy consumption)_y',
'Revenue',
'Population ages 05-09, male (% of male population)',
'Population, female (% of total population)',
'Population ages 20-24, male (% of male population)',
'Rural population growth (annual %)',
'Population ages 20-24, female (% of female population)',
'Population ages 15-19, male (% of male population)',
'Fertility rate, total (births per woman)',
'Prevalence of anemia among children (% of children under 5)',
'Agricultural raw materials imports (% of merchandise imports)',
'Capture fisheries production (metric tons)',
'Net imports [Imports - Exports - Bunkers] (petajoules)',
'Prevalence of anemia among pregnant women (%)', 
'Employment in agriculture, male (% of male employment) (modeled ILO estimate)',
'Immunization, measles (% of children ages 12-23 months)',
'Prevalence of anemia among women of reproductive age (% of women ages 15-49)',
'Prevalence of anemia among non-pregnant women (% of women ages 15-49)',
'Young age (0-14) dependency ratio (per 100 people ages 15-64)',
'Export unit value index (2000 = 100)',
'Old-age (65 and older) dependency ratio (per 100 people ages 15-64)',
'Infants lacking immunization, measles (% of one-year-olds)',
'Gross national expenditure deflator (base year varies by country)',
'General government final consumption expenditure (% of GDP)',
'School enrollment, primary, female (% gross)',
'Primary school starting age (years)',
'GDP (constant 2010 US$)',
'Inflation, average consumer prices',
'Gross domestic savings (current US$)',   
'Supply per capita (gigajoules)',
'Primary income payments (BoP, current US$)',
'Import volume index (2000 = 100)',
'Emissions per capita (metric tons of carbon dioxide)',
'Net primary income (BoP, current US$)',
'Final consumption expenditure (% of GDP)',
'Inflation, end of period consumer prices',
'Gross capital formation (% of GDP)_x',
'Gross capital formation (% of GDP)_y',
'Gross capital formation (annual % growth)',
'Gross capital formation (constant 2010 US$)',
'Gross capital formation (current US$)',
'Gross domestic product (GDP), total (2011 PPP $ billions)',
'Gross domestic product based on purchasing-power-parity (PPP) share of world total',
'Gross domestic product corresponding to fiscal year, current prices',
'Gross domestic product per capita, constant prices',
'Gross domestic product per capita, current prices',
'Gross domestic product, constant prices',
'Gross domestic product, current prices',
'Gross domestic product, deflator',
'Gross domestic savings (% of GDP)',
'Gross domestic savings (current US$)',
'Gross fixed capital formation (% of GDP)_x',
'Gross fixed capital formation (% of GDP)_y',
'Gross fixed capital formation (constant 2010 US$)',
'Gross fixed capital formation (current US$)',
'Gross intake ratio in first grade of primary education, total (% of relevant age group)',
'Gross national expenditure (% of GDP)',
'Gross national expenditure (constant 2010 US$)',
'Gross national expenditure (current US$)',
'Gross national expenditure deflator (base year varies by country)',
'Gross national income (GNI) per capita (2011 PPP $)',
'Gross savings (% of GDP)',
'Gross savings (% of GNI)',
'Gross savings (current US$)',
'Households and NPISHs Final consumption expenditure (annual % growth)',
'Households and NPISHs Final consumption expenditure (constant 2010 US$)',
'Households and NPISHs Final consumption expenditure (current US$)',
'Households and NPISHs Final consumption expenditure per capita (constant 2010 US$)',
'Households and NPISHs Final consumption expenditure per capita growth (annual %)',
'Households and NPISHs Final consumption expenditure, PPP (constant 2011 international $)',
'Households and NPISHs final consumption expenditure (% of GDP)',
'Domestic general government health expenditure (% of current health expenditure)',
'Domestic general government health expenditure (% of general government expenditure)',
'Domestic general government health expenditure per capita (current US$)',
'Domestic general government health expenditure (% of GDP)',
'Number of under-five deaths',
'Overall loss in HDI due to inequality (%)',
'Median age (years)',
'Human Development Index (HDI), male',
'Human Development Index (HDI), female',
'Life expectancy at birth, total (years)',
'Life expectancy at birth, male (years)',
'Life expectancy at birth, female (years)',
'Mean years of schooling, male (years)',
'Mean years of schooling, female (years)',
'Expected years of schooling, male (years)',
'Expected years of schooling, female (years)',
'Inequality-adjusted education index',
'Inequality in life expectancy (%)',
'Inequality-adjusted income index',
'Inequality-adjusted HDI (IHDI)',
'Inequality-adjusted life expectancy index',
'Gender Inequality Index (GII)'
]



### Human Development Index (HDI)
After evaluating a few different types of metrics, the metric I ended up using was the Human Development Index (HDI). It is not perfect but is pretty holistic / straight-forward and covers healthcare, education and income. For more details on how it is calculated, please go here: http://hdr.undp.org/sites/default/files/hdr2019_technical_notes.pdf

In [19]:
# Testing the HDI construction
life_expectancy = test['Life expectancy at birth (years)'].values[0]
print(life_expectancy)
expected_years = test['Expected years of schooling (years)'].values[0]
print (expected_years)
mean_years = test['Mean years of schooling (years)'].values[0]
print (mean_years)
gnipp = np.log(test['Gross national income (GNI) per capita (2011 PPP $)'].values[0])
print (np.exp(gnipp))
print (gnipp)

# Creating a function to calculate the HDI based on inputs)

def hdi_calculator (gnipp, life_expectancy, mean_years, expected_years):
    health_index = (life_expectancy - 20) / (85 - 20)
    expected_years_index = (expected_years - 0) / (18 - 0)
    mean_years_index = (mean_years - 0) / (15 - 0)
    education_index = (mean_years_index + expected_years_index) / 2
    #income_index = (np.log(gnipp) - np.log (100)) / (np.log(75000) - np.log (100))
    income_index = (gnipp - np.log (100)) / (np.log(75000) - np.log (100))
    hdi = (health_index * education_index * income_index)**(1/3)
    return hdi

hdi_calculator(gnipp, life_expectancy, mean_years, expected_years)

76.652
13.06962
9.29248
9838.802551999994
9.194089290771323


0.7407595767447375

In [20]:
# Building a model to isolate top predictors

# Creating the predictor matrix for train, dev and test sets
X_train = train.drop(columns = predictors_to_drop + target_indicators_all + ['country_code' , 'country', 'date'])
X_dev = dev.drop(columns = predictors_to_drop + target_indicators_all + ['country_code' , 'country', 'date'])
X_test = test.drop(columns = predictors_to_drop + target_indicators_all + ['country_code' , 'country', 'date'])

# For Timeseries Cross Validation. Since this data had time-bound data, this is an additional cross val test...
# ...to ensure that I am not very off
unique_years = work.date.unique()
X = work.drop(columns = predictors_to_drop + target_indicators_all + ['country_code' , 'country'])

# Initiating a StandardScaler object
scaler_X = StandardScaler()

### Model Election
At this point, I ran literally hundreds of different types of models trying to predict components of HDI (Mean Years of School, Expected Years of Schooling, GNIPP, Life Expectancy) as well trying to predict HDI as a final metric. 

Remember, in this step what was important was to able to isolate those indicators which were the strongest, least related and most controllable. It was important to be able to see feature importance / coefficient weights so I could only use model types that gave me those (Linear Regression, SVR Regression, Decision Tree Regressors including Boosting and Bagging Regressors). In the end, Linear Regression with Lasso regularlization was the most meaningful. That's the only one I show in this jupyter notebook for simplicitiy's sake.

In [21]:
#HDI Impact Checker - Lasso

# Isolating the target variables
target = 'Human Development Index (HDI)'

y_train_hdi = train[target]
y_dev_hdi = dev[target]
y_test_hdi = test[target]
y = work[['date'] + [target]]

# Best model after Grid searching and evaluating many models
model_hdi = LassoCV(alphas=np.logspace(-4, 4, 100), cv=5, fit_intercept=True, max_iter=10000, tol = 0.01)

# Building a pipe to make the process more efficient
pipe_hdi = Pipeline(steps=[
                       ('standardize', scaler_X),
                       ('model', model_hdi)])

# Fitting the model
pipe_hdi.fit (X_train,y_train_hdi)

# Putting the predictions in a dataframe
pred_hdi = pd.DataFrame(pipe_hdi.predict (X_dev))

# Kfold cross validation
kf = KFold(n_splits=5, shuffle=True, random_state=1)
scores_shuffled = cross_val_score(pipe_hdi, X_train, y_train_hdi, cv=kf)

# Time_series cross validation (since this data had time-bound data, this is an additional cross val test to ensure...
# ... that I am not very off.)
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

ts_scores_train = []
ts_scores_test = []

for train_index, test_index in tscv.split(unique_years):
    X_train_ts = X[X.date.isin (unique_years[train_index])]
    X_test_ts = X[X.date.isin (unique_years[test_index])]
    X_train_ts.pop ('date')
    X_test_ts.pop ('date')
    
    y_train_ts = y[y.date.isin (unique_years[train_index])]
    y_test_ts = y[y.date.isin (unique_years[test_index])]
    y_train_ts.pop ('date')
    y_test_ts.pop ('date')
    
    ts_scores_train.append(pipe_hdi.score (X_train_ts, y_train_ts))
    ts_scores_test.append(pipe_hdi.score (X_test_ts, y_test_ts))

# Printing all ther results
print("\nKfold Cross Validation:")
print("Shuffled Cross-validated scores:", scores_shuffled)
print("Mean of Shuffled Cross-validated scores:", scores_shuffled.mean())

print("\nTime Series Cross Validation:")
print("Train Time-Series Cross Val mean score:", np.mean(ts_scores_train))
print("Test Time-Series Cross Val mean score:", np.mean(ts_scores_test))

print("\nTraining Score:", pipe_hdi.score(X_train,y_train_hdi))
print("Dev Score:", pipe_hdi.score(X_dev,y_dev_hdi))
print("Test Score:", pipe_hdi.score(X_test,y_test_hdi))

print ("\nRoot of Mean Squared Error:", mean_squared_error (y_dev_hdi,pred_hdi)**0.5)

print ("\nNegative Values:", np.sum ((pred_hdi <= 0).values*1))

# Building a dataframe for coefficients to isolate which ones are the most impactful
df_coef = pd.DataFrame(list(zip(X_train.columns, pipe_hdi.named_steps['model'].coef_)))
df_coef.columns = ['predictors', 'coefficients']
df_coef['coef_abs'] = df_coef.coefficients.abs()
df_coef.sort_values (by='coef_abs', ascending = False, inplace=True)
df_coef[df_coef.coef_abs>0]


Kfold Cross Validation:
Shuffled Cross-validated scores: [0.97743594 0.97554263 0.97186961 0.97706662 0.97338434]
Mean of Shuffled Cross-validated scores: 0.9750598295837094

Time Series Cross Validation:
Train Time-Series Cross Val mean score: 0.9754504055315507
Test Time-Series Cross Val mean score: 0.9696889565118262

Training Score: 0.9794941116591005
Dev Score: 0.9752495398201875
Test Score: 0.9388380208680053

Root of Mean Squared Error: 0.015838631463742172

Negative Values: 0


,predictors,coefficients,coef_abs
119,Inequality in education (%),-0.0154,0.0154
208,"School enrollment, primary (% gross)",0.0098,0.0098
184,"Population ages 15-64, female (% of female pop...",0.0095,0.0095
35,Domestic general government health expenditure...,0.0093,0.0093
1,"Adolescent fertility rate (births per 1,000 wo...",-0.0090,0.0090
88,Forest area (sq. km),-0.0078,0.0078
216,"Secondary education, general pupils (% female)",0.0073,0.0073
7,"Agriculture, forestry, and fishing, value adde...",-0.0073,0.0073
174,"Out-of-pocket expenditure per capita, PPP (cur...",0.0072,0.0072
3,Agricultural land (% of land area),-0.0070,0.0070


In [22]:
# Exporting these coefficients to a csv so that I can manually adjust and look for onces that are most impactful
# Used MS Excel for this because it was a small dataset and more flexible for manual analysis
df_coef.to_csv('../raw_data/picking_predictors.csv', index=False)

## Step 2: Using top selected indicators (19) to build best model to predict HDI

In [24]:
# Based on the analysis done in Excel, these were top 19 indicators that were picked based on prediction power,...
# ... controllability and relatedness (lack of)

predictors_again = [
'Inequality in education (%)',
'School enrollment, primary (% gross)',
'Domestic general government health expenditure per capita, PPP (current international $)',
'Agriculture, forestry, and fishing, value added (% of GDP)',
'Adolescent fertility rate (births per 1,000 women ages 15-19)',
'Pupil-teacher ratio, primary',
'Tuberculosis incidence (per 100,000 people)',
'Secondary education, general pupils (% female)',
'Out-of-pocket expenditure per capita, PPP (current international $)',
'Inequality in income (%)',
'PM2.5 pollution, population exposed to levels exceeding WHO Interim Target-3 value (% of total)',
'Forest area (% of land area)',
'Mobile cellular subscriptions (per 100 people)',
'International inbound tourists (thousands)',
'Government expenditure on education, total (% of GDP)',
'Rural population with access to electricity (%)',
'Unemployment, total (% of labour force)',
'Corruption Perception Index',
'Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)',
 ]

In [25]:
# Now that we have the indicators, we have to rebuild the train, dev and test dataset
# The reason to rebuild these datasets is because we have lesser indicators, we should have more rows to work with

work_log = witer[['country_code', 'country', 'date'] + predictors_again + target_indicators_all].dropna()
print (work_log.shape)
work_log.head()

(3087, 30)


,country_code,country,date,Inequality in education (%),"School enrollment, primary (% gross)","Domestic general government health expenditure per capita, PPP (current international $)","Agriculture, forestry, and fishing, value added (% of GDP)","Adolescent fertility rate (births per 1,000 women ages 15-19)","Pupil-teacher ratio, primary","Tuberculosis incidence (per 100,000 people)","Secondary education, general pupils (% female)","Out-of-pocket expenditure per capita, PPP (current international $)",Inequality in income (%),"PM2.5 pollution, population exposed to levels exceeding WHO Interim Target-3 value (% of total)",Forest area (% of land area),Mobile cellular subscriptions (per 100 people),International inbound tourists (thousands),"Government expenditure on education, total (% of GDP)",Rural population with access to electricity (%),"Unemployment, total (% of labour force)",Corruption Perception Index,"Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)",Mean years of schooling (years),Life expectancy index,Income index,Education index,Human Development Index (HDI),Gross national income (GNI) per capita (2011 PPP $),Life expectancy at birth (years),Expected years of schooling (years)
0,AGO,Angola,1990,31.7946,73.0396,47.1677,7.6161,216.6000,36.4299,357.3732,38.3306,55.7882,40.7507,100.0000,48.9099,3.0946,158.5374,2.5995,7.5186,22.6543,19.3559,74.9450,4.4740,0.3338,0.4563,0.1906,0.3119,2050.0510,41.6960,3.7809
1,AGO,Angola,1991,31.5819,66.8044,45.0410,7.6014,214.8000,31.8767,354.7743,46.5205,55.6669,42.5613,100.0000,48.8098,-0.0675,151.8990,2.5995,11.7030,23.7000,19.3559,74.8790,4.4740,0.3362,0.4529,0.2017,0.3203,2005.3859,41.8550,3.5662
2,AGO,Angola,1992,31.4278,64.2471,53.9656,7.3658,213.0000,27.1810,327.4419,46.5205,53.5362,44.0560,100.0000,48.7097,0.7188,136.2831,2.5995,11.7030,23.3722,19.3559,74.8260,4.4386,0.3394,0.4498,0.2044,0.3195,1963.6370,42.0600,3.5471
3,AGO,Angola,1993,31.7154,71.9530,50.7740,7.4091,210.9000,36.6296,322.2027,46.5205,52.8076,44.1965,100.0000,48.6096,0.0084,121.4084,2.5995,11.7030,23.7876,19.3559,74.7820,4.4819,0.3435,0.4028,0.2186,0.3267,1439.3732,42.3290,3.6570
4,AGO,Angola,1994,31.3794,69.2585,54.2890,7.3115,208.8000,36.8101,288.0689,46.5205,51.2472,42.1421,100.0000,48.5095,0.0135,97.4764,2.5995,11.7030,24.2562,19.3559,74.7700,4.4571,0.3489,0.3625,0.2288,0.3310,1102.4029,42.6770,3.7668


In [26]:
# Removing any negative values that might have been caused by imputation
work_log['negative_vals'] = work_log[work_log[predictors_again] < 0].count(axis=1) >= 1
work_log = work_log[work_log.negative_vals == False]
work_log.pop('negative_vals')
print (work_log.shape)
work_log.head()

(2882, 30)


,country_code,country,date,Inequality in education (%),"School enrollment, primary (% gross)","Domestic general government health expenditure per capita, PPP (current international $)","Agriculture, forestry, and fishing, value added (% of GDP)","Adolescent fertility rate (births per 1,000 women ages 15-19)","Pupil-teacher ratio, primary","Tuberculosis incidence (per 100,000 people)","Secondary education, general pupils (% female)","Out-of-pocket expenditure per capita, PPP (current international $)",Inequality in income (%),"PM2.5 pollution, population exposed to levels exceeding WHO Interim Target-3 value (% of total)",Forest area (% of land area),Mobile cellular subscriptions (per 100 people),International inbound tourists (thousands),"Government expenditure on education, total (% of GDP)",Rural population with access to electricity (%),"Unemployment, total (% of labour force)",Corruption Perception Index,"Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)",Mean years of schooling (years),Life expectancy index,Income index,Education index,Human Development Index (HDI),Gross national income (GNI) per capita (2011 PPP $),Life expectancy at birth (years),Expected years of schooling (years)
0,AGO,Angola,1990,31.7946,73.0396,47.1677,7.6161,216.6000,36.4299,357.3732,38.3306,55.7882,40.7507,100.0000,48.9099,3.0946,158.5374,2.5995,7.5186,22.6543,19.3559,74.9450,4.4740,0.3338,0.4563,0.1906,0.3119,2050.0510,41.6960,3.7809
2,AGO,Angola,1992,31.4278,64.2471,53.9656,7.3658,213.0000,27.1810,327.4419,46.5205,53.5362,44.0560,100.0000,48.7097,0.7188,136.2831,2.5995,11.7030,23.3722,19.3559,74.8260,4.4386,0.3394,0.4498,0.2044,0.3195,1963.6370,42.0600,3.5471
3,AGO,Angola,1993,31.7154,71.9530,50.7740,7.4091,210.9000,36.6296,322.2027,46.5205,52.8076,44.1965,100.0000,48.6096,0.0084,121.4084,2.5995,11.7030,23.7876,19.3559,74.7820,4.4819,0.3435,0.4028,0.2186,0.3267,1439.3732,42.3290,3.6570
4,AGO,Angola,1994,31.3794,69.2585,54.2890,7.3115,208.8000,36.8101,288.0689,46.5205,51.2472,42.1421,100.0000,48.5095,0.0135,97.4764,2.5995,11.7030,24.2562,19.3559,74.7700,4.4571,0.3489,0.3625,0.2288,0.3310,1102.4029,42.6770,3.7668
5,AGO,Angola,1995,31.1946,72.6726,62.9207,6.6532,206.7000,36.0369,330.1402,46.5205,50.5284,38.7731,100.0000,48.4094,0.0143,9.0000,2.5995,11.3216,23.9000,19.3559,74.7840,4.4203,0.3558,0.4432,0.2397,0.3424,1879.7079,43.1250,3.8767


In [27]:
# Taking a deeper look at the indicators
work_log.describe()

,date,Inequality in education (%),"School enrollment, primary (% gross)","Domestic general government health expenditure per capita, PPP (current international $)","Agriculture, forestry, and fishing, value added (% of GDP)","Adolescent fertility rate (births per 1,000 women ages 15-19)","Pupil-teacher ratio, primary","Tuberculosis incidence (per 100,000 people)","Secondary education, general pupils (% female)","Out-of-pocket expenditure per capita, PPP (current international $)",Inequality in income (%),"PM2.5 pollution, population exposed to levels exceeding WHO Interim Target-3 value (% of total)",Forest area (% of land area),Mobile cellular subscriptions (per 100 people),International inbound tourists (thousands),"Government expenditure on education, total (% of GDP)",Rural population with access to electricity (%),"Unemployment, total (% of labour force)",Corruption Perception Index,"Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)",Mean years of schooling (years),Life expectancy index,Income index,Education index,Human Development Index (HDI),Gross national income (GNI) per capita (2011 PPP $),Life expectancy at birth (years),Expected years of schooling (years)
count,2882.0000,2882.0000,2882.0000,2882.0000,2882.0000,2882.0000,2882.0000,2882.0000,2882.0000,2882.0000,2882.0000,2882.0000,2882.0000,2882.0000,2882.0000,2882.0000,2882.0000,2882.0000,2882.0000,2882.0000,2882.0000,2882.0000,2882.0000,2882.0000,2882.0000,2882.0000,2882.0000,2882.0000
mean,2004.2939,19.4282,101.4183,556.4133,13.6658,62.1762,27.3282,144.7080,48.1923,223.3137,23.9251,80.2958,30.7278,51.0951,5742.8847,4.3553,68.8732,8.3256,40.1681,52.9205,7.5698,0.7434,0.6447,0.5856,0.6506,12498.8970,68.3178,12.0232
std,7.8741,14.1350,15.6208,865.5722,11.8502,48.1762,14.0593,189.6586,5.1625,248.3534,11.2547,30.9595,20.1178,49.8769,11461.8888,2.3693,37.6367,6.1461,19.1728,15.2352,3.1710,0.1482,0.1728,0.1872,0.1646,12510.0404,9.6315,3.2972
min,1990.0000,0.2341,27.1665,0.2616,0.2472,1.3790,8.6800,1.3828,23.4413,2.3366,4.1206,0.0000,0.0442,0.0001,6.4262,0.6595,0.0383,0.2000,4.0000,10.3660,0.7144,0.1171,0.2387,0.0813,0.1987,485.6638,27.6100,2.0705
25%,1998.0000,6.6014,97.0145,38.6922,4.0122,21.0746,17.0448,20.9024,46.9518,52.2794,16.1000,69.5705,12.5751,2.7867,407.5904,3.1352,34.0737,4.0881,26.5055,44.7007,5.0001,0.6545,0.5086,0.4488,0.5260,2898.7138,62.5415,10.1595
50%,2004.5000,16.7830,102.3198,175.7629,9.7347,50.9860,23.1274,68.5515,49.9656,136.9209,21.0350,99.4757,30.6759,33.6102,1372.2678,4.2307,92.2382,6.8496,34.0000,52.1720,7.8703,0.7790,0.6580,0.6138,0.6707,7793.7925,70.6380,12.2158
75%,2011.0000,31.1000,108.2184,644.1218,20.8561,90.6763,35.1823,202.7478,51.1307,300.5056,29.7349,100.0000,45.2203,97.6073,5723.3895,5.1857,100.0000,10.8958,49.0000,62.4265,10.2075,0.8534,0.7838,0.7266,0.7786,17925.0515,75.4735,14.4191
max,2017.0000,74.4870,165.6454,7366.7947,60.2835,223.2282,82.7954,1280.0000,59.9085,2325.7402,68.9637,101.8389,85.3805,182.4573,84452.0000,62.9117,107.5989,37.3000,100.0000,90.3450,14.0820,0.9765,0.9762,0.9413,0.9440,64050.6176,83.4730,23.2822


In [28]:
# Analyzing skew in the predictors so we can smooth the data as needed
work_log[predictors_again].skew()

Inequality in education (%)                                                                        0.5194
School enrollment, primary (% gross)                                                              -0.8834
Domestic general government health expenditure per capita, PPP (current international $)           2.5502
Agriculture, forestry, and fishing, value added (% of GDP)                                         1.1018
Adolescent fertility rate (births per 1,000 women ages 15-19)                                      0.8870
Pupil-teacher ratio, primary                                                                       1.0761
Tuberculosis incidence (per 100,000 people)                                                        2.4071
Secondary education, general pupils (% female)                                                    -1.8545
Out-of-pocket expenditure per capita, PPP (current international $)                                2.4811
Inequality in income (%)                      

In [29]:
# Logging columns that have a skew greater than 1
logged_columns = []
for predictor in predictors_again:
    if np.abs(work_log[predictor].skew()) >=1:
        if np.abs(np.log(work_log[predictor]).skew()) < np.abs(work_log[predictor].skew()):
            work_log[predictor] = np.log(work_log[predictor])
            logged_columns.append(predictor)

# As the output shows, the data is much less skewed after logging the relevant columns 
work_log[predictors_again].skew()

Inequality in education (%)                                                                        0.5194
School enrollment, primary (% gross)                                                              -0.8834
Domestic general government health expenditure per capita, PPP (current international $)          -0.2306
Agriculture, forestry, and fishing, value added (% of GDP)                                        -0.4852
Adolescent fertility rate (births per 1,000 women ages 15-19)                                      0.8870
Pupil-teacher ratio, primary                                                                       0.1376
Tuberculosis incidence (per 100,000 people)                                                       -0.1343
Secondary education, general pupils (% female)                                                    -1.8545
Out-of-pocket expenditure per capita, PPP (current international $)                               -0.3527
Inequality in income (%)                      

In [30]:
# Recreating the test / train / dev sets based on true values and iterated values as before
true_values = woiter[work_log.columns].dropna()
true_values = work_log[work_log.index.isin (true_values.index)]
print (true_values.shape)
true_values.head()

(405, 30)


,country_code,country,date,Inequality in education (%),"School enrollment, primary (% gross)","Domestic general government health expenditure per capita, PPP (current international $)","Agriculture, forestry, and fishing, value added (% of GDP)","Adolescent fertility rate (births per 1,000 women ages 15-19)","Pupil-teacher ratio, primary","Tuberculosis incidence (per 100,000 people)","Secondary education, general pupils (% female)","Out-of-pocket expenditure per capita, PPP (current international $)",Inequality in income (%),"PM2.5 pollution, population exposed to levels exceeding WHO Interim Target-3 value (% of total)",Forest area (% of land area),Mobile cellular subscriptions (per 100 people),International inbound tourists (thousands),"Government expenditure on education, total (% of GDP)",Rural population with access to electricity (%),"Unemployment, total (% of labour force)",Corruption Perception Index,"Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)",Mean years of schooling (years),Life expectancy index,Income index,Education index,Human Development Index (HDI),Gross national income (GNI) per capita (2011 PPP $),Life expectancy at birth (years),Expected years of schooling (years)
20,AGO,Angola,2010,26.2000,105.7810,4.5875,1.8214,172.9316,3.8196,5.9506,46.2789,3.4209,3.5946,100.0000,46.9078,40.2606,6.0521,1.2300,16.2100,2.2925,2.9444,75.3070,4.7264,0.5876,0.6031,0.3977,0.5204,5420.6528,58.1920,8.6462
51,ALB,Albania,2013,11.9000,99.9839,5.6704,2.9738,20.5124,2.9508,2.9444,48.8104,5.9050,2.9069,94.2903,28.2270,126.9370,7.9575,1.2639,100.0000,2.7473,3.4340,44.6500,9.6578,0.8877,0.7051,0.7323,0.7710,10648.3919,77.7020,14.7729
53,ALB,Albania,2015,11.9000,105.5430,5.7764,2.9847,20.0772,2.9240,2.7726,49.2511,6.0634,2.9086,93.2226,28.1569,117.6592,8.2385,1.2349,100.0000,2.8391,3.5835,47.6760,9.6798,0.8950,0.7112,0.7330,0.7756,11083.4795,78.1740,14.7729
54,ALB,Albania,2016,9.0840,105.5539,5.7499,2.9882,19.8596,2.9013,2.7726,49.5154,6.0878,2.9086,83.4325,28.1219,116.7444,8.3114,1.3749,100.0000,2.7213,3.6636,47.4590,9.9576,0.8976,0.7169,0.7423,0.7817,11512.3078,78.3450,14.7729
160,AUT,Austria,2010,2.4000,99.7898,8.0442,0.2369,9.5292,2.4005,2.2083,50.2718,6.6853,2.7147,52.0534,46.7476,145.5538,9.9990,1.7406,100.0000,1.5686,4.3694,53.5070,11.7838,0.9321,0.9184,0.8372,0.8949,43700.0727,80.5840,15.9985


In [31]:
iterated_values = work_log[~work_log.index.isin (true_values.index)]
print (iterated_values.shape)
iterated_values.head()

(2477, 30)


,country_code,country,date,Inequality in education (%),"School enrollment, primary (% gross)","Domestic general government health expenditure per capita, PPP (current international $)","Agriculture, forestry, and fishing, value added (% of GDP)","Adolescent fertility rate (births per 1,000 women ages 15-19)","Pupil-teacher ratio, primary","Tuberculosis incidence (per 100,000 people)","Secondary education, general pupils (% female)","Out-of-pocket expenditure per capita, PPP (current international $)",Inequality in income (%),"PM2.5 pollution, population exposed to levels exceeding WHO Interim Target-3 value (% of total)",Forest area (% of land area),Mobile cellular subscriptions (per 100 people),International inbound tourists (thousands),"Government expenditure on education, total (% of GDP)",Rural population with access to electricity (%),"Unemployment, total (% of labour force)",Corruption Perception Index,"Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)",Mean years of schooling (years),Life expectancy index,Income index,Education index,Human Development Index (HDI),Gross national income (GNI) per capita (2011 PPP $),Life expectancy at birth (years),Expected years of schooling (years)
0,AGO,Angola,1990,31.7946,73.0396,3.8537,2.0303,216.6000,3.5954,5.8788,38.3306,4.0216,3.7075,100.0000,48.9099,3.0946,5.0660,0.9553,7.5186,3.1203,2.9630,74.9450,4.4740,0.3338,0.4563,0.1906,0.3119,2050.0510,41.6960,3.7809
2,AGO,Angola,1992,31.4278,64.2471,3.9883,1.9968,213.0000,3.3025,5.7913,46.5205,3.9804,3.7855,100.0000,48.7097,0.7188,4.9147,0.9553,11.7030,3.1515,2.9630,74.8260,4.4386,0.3394,0.4498,0.2044,0.3195,1963.6370,42.0600,3.5471
3,AGO,Angola,1993,31.7154,71.9530,3.9274,2.0027,210.9000,3.6009,5.7752,46.5205,3.9667,3.7886,100.0000,48.6096,0.0084,4.7992,0.9553,11.7030,3.1692,2.9630,74.7820,4.4819,0.3435,0.4028,0.2186,0.3267,1439.3732,42.3290,3.6570
4,AGO,Angola,1994,31.3794,69.2585,3.9943,1.9894,208.8000,3.6058,5.6632,46.5205,3.9367,3.7410,100.0000,48.5095,0.0135,4.5796,0.9553,11.7030,3.1887,2.9630,74.7700,4.4571,0.3489,0.3625,0.2288,0.3310,1102.4029,42.6770,3.7668
5,AGO,Angola,1995,31.1946,72.6726,4.1419,1.8951,206.7000,3.5845,5.7995,46.5205,3.9225,3.6577,100.0000,48.4094,0.0143,2.1972,0.9553,11.3216,3.1739,2.9630,74.7840,4.4203,0.3558,0.4432,0.2397,0.3424,1879.7079,43.1250,3.8767


In [32]:
true_values.date.value_counts()

2013    64
2011    64
2010    60
2014    58
2015    56
2012    56
2016    47
Name: date, dtype: int64

In [33]:
dev_true = true_values[true_values.date.isin (['2010','2012', '2014'])]
print (dev_true.shape)
dev_true.head()

(174, 30)


,country_code,country,date,Inequality in education (%),"School enrollment, primary (% gross)","Domestic general government health expenditure per capita, PPP (current international $)","Agriculture, forestry, and fishing, value added (% of GDP)","Adolescent fertility rate (births per 1,000 women ages 15-19)","Pupil-teacher ratio, primary","Tuberculosis incidence (per 100,000 people)","Secondary education, general pupils (% female)","Out-of-pocket expenditure per capita, PPP (current international $)",Inequality in income (%),"PM2.5 pollution, population exposed to levels exceeding WHO Interim Target-3 value (% of total)",Forest area (% of land area),Mobile cellular subscriptions (per 100 people),International inbound tourists (thousands),"Government expenditure on education, total (% of GDP)",Rural population with access to electricity (%),"Unemployment, total (% of labour force)",Corruption Perception Index,"Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)",Mean years of schooling (years),Life expectancy index,Income index,Education index,Human Development Index (HDI),Gross national income (GNI) per capita (2011 PPP $),Life expectancy at birth (years),Expected years of schooling (years)
20,AGO,Angola,2010,26.2000,105.7810,4.5875,1.8214,172.9316,3.8196,5.9506,46.2789,3.4209,3.5946,100.0000,46.9078,40.2606,6.0521,1.2300,16.2100,2.2925,2.9444,75.3070,4.7264,0.5876,0.6031,0.3977,0.5204,5420.6528,58.1920,8.6462
160,AUT,Austria,2010,2.4000,99.7898,8.0442,0.2369,9.5292,2.4005,2.2083,50.2718,6.6853,2.7147,52.0534,46.7476,145.5538,9.9990,1.7406,100.0000,1.5686,4.3694,53.5070,11.7838,0.9321,0.9184,0.8372,0.8949,43700.0727,80.5840,15.9985
162,AUT,Austria,2012,2.5000,100.5991,8.1496,0.2990,8.4640,2.3818,2.1282,50.2892,6.7880,2.5416,39.8091,46.8009,159.8170,10.0921,1.7012,100.0000,1.5892,4.2341,54.3370,11.8216,0.9372,0.9215,0.8405,0.8987,44597.3880,80.9180,16.0732
164,AUT,Austria,2014,3.5000,101.6282,8.2122,0.1829,8.0144,2.3719,2.0149,49.8886,6.8780,2.7408,23.5264,46.8621,150.3457,10.1382,1.6952,100.0000,1.7228,4.2767,54.5620,12.1487,0.9423,0.9207,0.8444,0.9015,44379.5683,81.2500,15.8183
188,AZE,Azerbaijan,2010,12.0000,93.7623,5.2058,1.7128,44.9380,2.4017,4.6444,46.6928,6.3597,1.4816,100.0000,12.1988,100.7490,7.1546,1.0227,99.9506,1.7228,3.1781,60.2230,10.7189,0.7844,0.7593,0.6809,0.7402,15246.1256,70.9870,11.6505


In [34]:
iterated_values.date.value_counts()

2017    114
2005    111
2000    109
2008    107
2002    107
2009    107
2007    107
2003    107
2004    106
1998    105
2006    105
1999    105
2001    105
1997    101
1996     99
1995     96
1994     88
1993     83
1992     80
1991     79
1990     71
2016     67
2014     58
2015     57
2010     53
2012     53
2013     50
2011     47
Name: date, dtype: int64

In [35]:
dev_iterated = iterated_values[iterated_values.date.isin (['2017', '2013', '2011'])]
print (dev_iterated.shape)

(211, 30)


In [36]:
dev_log = pd.concat([dev_true,dev_iterated])
print (dev_log.shape)
dev_log.head()

(385, 30)


,country_code,country,date,Inequality in education (%),"School enrollment, primary (% gross)","Domestic general government health expenditure per capita, PPP (current international $)","Agriculture, forestry, and fishing, value added (% of GDP)","Adolescent fertility rate (births per 1,000 women ages 15-19)","Pupil-teacher ratio, primary","Tuberculosis incidence (per 100,000 people)","Secondary education, general pupils (% female)","Out-of-pocket expenditure per capita, PPP (current international $)",Inequality in income (%),"PM2.5 pollution, population exposed to levels exceeding WHO Interim Target-3 value (% of total)",Forest area (% of land area),Mobile cellular subscriptions (per 100 people),International inbound tourists (thousands),"Government expenditure on education, total (% of GDP)",Rural population with access to electricity (%),"Unemployment, total (% of labour force)",Corruption Perception Index,"Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)",Mean years of schooling (years),Life expectancy index,Income index,Education index,Human Development Index (HDI),Gross national income (GNI) per capita (2011 PPP $),Life expectancy at birth (years),Expected years of schooling (years)
20,AGO,Angola,2010,26.2000,105.7810,4.5875,1.8214,172.9316,3.8196,5.9506,46.2789,3.4209,3.5946,100.0000,46.9078,40.2606,6.0521,1.2300,16.2100,2.2925,2.9444,75.3070,4.7264,0.5876,0.6031,0.3977,0.5204,5420.6528,58.1920,8.6462
160,AUT,Austria,2010,2.4000,99.7898,8.0442,0.2369,9.5292,2.4005,2.2083,50.2718,6.6853,2.7147,52.0534,46.7476,145.5538,9.9990,1.7406,100.0000,1.5686,4.3694,53.5070,11.7838,0.9321,0.9184,0.8372,0.8949,43700.0727,80.5840,15.9985
162,AUT,Austria,2012,2.5000,100.5991,8.1496,0.2990,8.4640,2.3818,2.1282,50.2892,6.7880,2.5416,39.8091,46.8009,159.8170,10.0921,1.7012,100.0000,1.5892,4.2341,54.3370,11.8216,0.9372,0.9215,0.8405,0.8987,44597.3880,80.9180,16.0732
164,AUT,Austria,2014,3.5000,101.6282,8.2122,0.1829,8.0144,2.3719,2.0149,49.8886,6.8780,2.7408,23.5264,46.8621,150.3457,10.1382,1.6952,100.0000,1.7228,4.2767,54.5620,12.1487,0.9423,0.9207,0.8444,0.9015,44379.5683,81.2500,15.8183
188,AZE,Azerbaijan,2010,12.0000,93.7623,5.2058,1.7128,44.9380,2.4017,4.6444,46.6928,6.3597,1.4816,100.0000,12.1988,100.7490,7.1546,1.0227,99.9506,1.7228,3.1781,60.2230,10.7189,0.7844,0.7593,0.6809,0.7402,15246.1256,70.9870,11.6505


In [37]:
test_log = true_values[~true_values.index.isin (dev_true.index)]
print (test_log.shape)
test_log.head()

(231, 30)


,country_code,country,date,Inequality in education (%),"School enrollment, primary (% gross)","Domestic general government health expenditure per capita, PPP (current international $)","Agriculture, forestry, and fishing, value added (% of GDP)","Adolescent fertility rate (births per 1,000 women ages 15-19)","Pupil-teacher ratio, primary","Tuberculosis incidence (per 100,000 people)","Secondary education, general pupils (% female)","Out-of-pocket expenditure per capita, PPP (current international $)",Inequality in income (%),"PM2.5 pollution, population exposed to levels exceeding WHO Interim Target-3 value (% of total)",Forest area (% of land area),Mobile cellular subscriptions (per 100 people),International inbound tourists (thousands),"Government expenditure on education, total (% of GDP)",Rural population with access to electricity (%),"Unemployment, total (% of labour force)",Corruption Perception Index,"Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)",Mean years of schooling (years),Life expectancy index,Income index,Education index,Human Development Index (HDI),Gross national income (GNI) per capita (2011 PPP $),Life expectancy at birth (years),Expected years of schooling (years)
51,ALB,Albania,2013,11.9000,99.9839,5.6704,2.9738,20.5124,2.9508,2.9444,48.8104,5.9050,2.9069,94.2903,28.2270,126.9370,7.9575,1.2639,100.0000,2.7473,3.4340,44.6500,9.6578,0.8877,0.7051,0.7323,0.7710,10648.3919,77.7020,14.7729
53,ALB,Albania,2015,11.9000,105.5430,5.7764,2.9847,20.0772,2.9240,2.7726,49.2511,6.0634,2.9086,93.2226,28.1569,117.6592,8.2385,1.2349,100.0000,2.8391,3.5835,47.6760,9.6798,0.8950,0.7112,0.7330,0.7756,11083.4795,78.1740,14.7729
54,ALB,Albania,2016,9.0840,105.5539,5.7499,2.9882,19.8596,2.9013,2.7726,49.5154,6.0878,2.9086,83.4325,28.1219,116.7444,8.3114,1.3749,100.0000,2.7213,3.6636,47.4590,9.9576,0.8976,0.7169,0.7423,0.7817,11512.3078,78.3450,14.7729
161,AUT,Austria,2011,2.4000,99.7387,8.0852,0.3378,8.9966,2.3882,2.2083,50.2482,6.7296,2.7147,55.1546,46.7751,154.0495,10.0438,1.7214,100.0000,1.5261,4.3550,53.8390,11.7802,0.9347,0.9215,0.8379,0.8970,44602.6110,80.7530,16.0282
163,AUT,Austria,2013,3.7000,101.2101,8.1801,0.2269,8.2392,2.3706,2.1282,49.9653,6.8545,2.6247,36.2748,46.8357,155.1158,10.1191,1.7133,100.0000,1.6677,4.2341,54.5990,11.8589,0.9397,0.9209,0.8333,0.8968,44420.2678,81.0830,15.7692


In [38]:
train_log = work_log[~(work_log.index.isin (test_log.index)) & ~(work_log.index.isin (dev_log.index))]
print (train_log.shape)
train_log.head()

(2266, 30)


,country_code,country,date,Inequality in education (%),"School enrollment, primary (% gross)","Domestic general government health expenditure per capita, PPP (current international $)","Agriculture, forestry, and fishing, value added (% of GDP)","Adolescent fertility rate (births per 1,000 women ages 15-19)","Pupil-teacher ratio, primary","Tuberculosis incidence (per 100,000 people)","Secondary education, general pupils (% female)","Out-of-pocket expenditure per capita, PPP (current international $)",Inequality in income (%),"PM2.5 pollution, population exposed to levels exceeding WHO Interim Target-3 value (% of total)",Forest area (% of land area),Mobile cellular subscriptions (per 100 people),International inbound tourists (thousands),"Government expenditure on education, total (% of GDP)",Rural population with access to electricity (%),"Unemployment, total (% of labour force)",Corruption Perception Index,"Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)",Mean years of schooling (years),Life expectancy index,Income index,Education index,Human Development Index (HDI),Gross national income (GNI) per capita (2011 PPP $),Life expectancy at birth (years),Expected years of schooling (years)
0,AGO,Angola,1990,31.7946,73.0396,3.8537,2.0303,216.6000,3.5954,5.8788,38.3306,4.0216,3.7075,100.0000,48.9099,3.0946,5.0660,0.9553,7.5186,3.1203,2.9630,74.9450,4.4740,0.3338,0.4563,0.1906,0.3119,2050.0510,41.6960,3.7809
2,AGO,Angola,1992,31.4278,64.2471,3.9883,1.9968,213.0000,3.3025,5.7913,46.5205,3.9804,3.7855,100.0000,48.7097,0.7188,4.9147,0.9553,11.7030,3.1515,2.9630,74.8260,4.4386,0.3394,0.4498,0.2044,0.3195,1963.6370,42.0600,3.5471
3,AGO,Angola,1993,31.7154,71.9530,3.9274,2.0027,210.9000,3.6009,5.7752,46.5205,3.9667,3.7886,100.0000,48.6096,0.0084,4.7992,0.9553,11.7030,3.1692,2.9630,74.7820,4.4819,0.3435,0.4028,0.2186,0.3267,1439.3732,42.3290,3.6570
4,AGO,Angola,1994,31.3794,69.2585,3.9943,1.9894,208.8000,3.6058,5.6632,46.5205,3.9367,3.7410,100.0000,48.5095,0.0135,4.5796,0.9553,11.7030,3.1887,2.9630,74.7700,4.4571,0.3489,0.3625,0.2288,0.3310,1102.4029,42.6770,3.7668
5,AGO,Angola,1995,31.1946,72.6726,4.1419,1.8951,206.7000,3.5845,5.7995,46.5205,3.9225,3.6577,100.0000,48.4094,0.0143,2.1972,0.9553,11.3216,3.1739,2.9630,74.7840,4.4203,0.3558,0.4432,0.2397,0.3424,1879.7079,43.1250,3.8767


In [39]:
# Ensuring the train, dev and test sets are all aligned and there are no duplicates
print (train_log.shape)
print (dev_log.shape)
print (test_log.shape)
print (work_log.shape)

(2266, 30)
(385, 30)
(231, 30)
(2882, 30)


In [40]:
# Creating predictor matrices for all the train, dev and test datasets
X_trimmed_train = train_log[predictors_again]
X_trimmed_dev = dev_log[predictors_again]
X_trimmed_test = test_log[predictors_again]
X_trimmed = work_log[predictors_again + ['date']]

In [41]:
# Ensuring there are no null values
X_trimmed_train.isnull().sum()

Inequality in education (%)                                                                        0
School enrollment, primary (% gross)                                                               0
Domestic general government health expenditure per capita, PPP (current international $)           0
Agriculture, forestry, and fishing, value added (% of GDP)                                         0
Adolescent fertility rate (births per 1,000 women ages 15-19)                                      0
Pupil-teacher ratio, primary                                                                       0
Tuberculosis incidence (per 100,000 people)                                                        0
Secondary education, general pupils (% female)                                                     0
Out-of-pocket expenditure per capita, PPP (current international $)                                0
Inequality in income (%)                                                                   

### Model Selection & Design:
For the final algorithm structure, the idea was predict the components of HDI (Mean Years of School, Expected Years of Schooling, GNIPP, Life Expectancy), and then use those predictions to calculate HDI based on the formula as per the UNDP formulation (http://hdr.undp.org/sites/default/files/hdr2019_technical_notes.pdf).

The goal was to eventually transfer this model to Tableau so I could create a toggle-able dashboard. Since I didnot have Tableau Server, I could not use TabPy and was only able to use Linear Regression. Why? Because Linear Regression gives me coefficients (and basically a formula) which I could then take to Tableau and use the model in the form of equations. 

Again, I toyed with many types of linear regression models and gridsearched all the four separate models of the four component indicators I was trying to predict to arrive at the best models which are presented below.

In [43]:
# Model for GNI PP - LASSO
target = 'Gross national income (GNI) per capita (2011 PPP $)'

y_train_gni = np.log(train_log[target])
y_dev_gni = np.log(dev_log[target])
y_test_gni = np.log(test_log[target])
y = work_log[['date'] + [target]]
y[target] = np.log(y[target].values)

model_gni = LassoCV(alphas=np.logspace(-4, 4, 100), cv=5, eps=10, fit_intercept=True,
                         max_iter=100, n_alphas=10, n_jobs=None,
                         normalize=False, positive=False, precompute='auto',
                         random_state=None, selection='cyclic', tol=0.001,
                         verbose=False)

pipe_gni = Pipeline(steps=[
                       ('standardize', scaler_X),
                       ('model', model_gni)])

pipe_gni.fit (X_trimmed_train,y_train_gni)

pred_gni = pd.DataFrame(pipe_gni.predict (X_trimmed_dev))

#kfold cross validation
kf = KFold(n_splits=5, shuffle=True, random_state=1)
scores_shuffled = cross_val_score(pipe_gni, X_trimmed_train, y_train_gni, cv=kf)

#time_series cross validation
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

ts_scores_train = []
ts_scores_test = []

for train_index, test_index in tscv.split(unique_years):
    X_trimmed_train_ts = X_trimmed[X_trimmed.date.isin (unique_years[train_index])]
    X_trimmed_test_ts = X_trimmed[X_trimmed.date.isin (unique_years[test_index])]
    X_trimmed_train_ts.pop ('date')
    X_trimmed_test_ts.pop ('date')
    
    y_train_ts = y[y.date.isin (unique_years[train_index])]
    y_test_ts = y[y.date.isin (unique_years[test_index])]
    y_train_ts.pop ('date')
    y_test_ts.pop ('date')
    
    ts_scores_train.append(pipe_gni.score (X_trimmed_train_ts, y_train_ts))
    ts_scores_test.append(pipe_gni.score (X_trimmed_test_ts, y_test_ts))

print("\nKfold Cross Validation:")
print("Shuffled Cross-validated scores:", scores_shuffled)
print("Mean of Shuffled Cross-validated scores:", scores_shuffled.mean())

print("\nTime Series Cross Validation:")
print("Train Time-Series Cross Val mean score:", np.mean(ts_scores_train))
print("Test Time-Series Cross Val mean score:", np.mean(ts_scores_test))

print("\nTraining Score:", pipe_gni.score(X_trimmed_train,y_train_gni))
print("Dev Score:", pipe_gni.score(X_trimmed_dev,y_dev_gni))
print("Test Score:", pipe_gni.score(X_trimmed_test,y_test_gni))

print ("\nRoot of Mean Squared Error:", mean_squared_error (y_dev_gni,pred_gni)**0.5)

print ("\nNegative Values:", np.sum ((pred_gni <= 0).values*1))

# Building a dataframe for coefficients
df_coef = pd.DataFrame(list(zip(X_trimmed_train.columns, pipe_gni.named_steps['model'].coef_)))
df_coef.columns = ['predictors', 'coefficients']
df_coef['coef_abs'] = df_coef.coefficients.abs()
df_coef.sort_values (by='coef_abs', ascending = False, inplace=True)
df_coef[df_coef.coef_abs>0]


Kfold Cross Validation:
Shuffled Cross-validated scores: [0.94203472 0.9478797  0.94801944 0.95092655 0.94897848]
Mean of Shuffled Cross-validated scores: 0.9475677782077472

Time Series Cross Validation:
Train Time-Series Cross Val mean score: 0.9405930356951959
Test Time-Series Cross Val mean score: 0.9572590590974656

Training Score: 0.9486268059511287
Dev Score: 0.9518251264431037
Test Score: 0.9598762484584898

Root of Mean Squared Error: 0.24870395953548904

Negative Values: 0


,predictors,coefficients,coef_abs
2,Domestic general government health expenditure...,0.4404,0.4404
3,"Agriculture, forestry, and fishing, value adde...",-0.2665,0.2665
8,"Out-of-pocket expenditure per capita, PPP (cur...",0.1450,0.1450
15,Rural population with access to electricity (%),0.1151,0.1151
4,"Adolescent fertility rate (births per 1,000 wo...",-0.0828,0.0828
13,International inbound tourists (thousands),0.0793,0.0793
17,Corruption Perception Index,0.0545,0.0545
10,"PM2.5 pollution, population exposed to levels ...",-0.0502,0.0502
14,"Government expenditure on education, total (% ...",-0.0501,0.0501
18,"Labor force participation rate, female (% of f...",-0.0465,0.0465


In [44]:
# Model for Life Expectancy - LASSO
target = 'Life expectancy at birth (years)'

y_train_le = train_log[target]
y_dev_le = dev_log[target]
y_test_le = test_log[target]
y = work_log[['date'] + [target]]

model_le = LassoCV(alphas=np.logspace(-4, 4, 100), cv=5, fit_intercept=True, max_iter=10000, tol = 0.01)

pipe_le = Pipeline(steps=[
                       ('standardize', scaler_X),
                       ('model', model_le)])

pipe_le.fit (X_trimmed_train,y_train_le)

pred_le = pd.DataFrame(pipe_le.predict (X_trimmed_dev))

#kfold cross validation
kf = KFold(n_splits=5, shuffle=True, random_state=1)
scores_shuffled = cross_val_score(pipe_le, X_trimmed_train, y_train_le, cv=kf)

#time_series cross validation
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

ts_scores_train = []
ts_scores_test = []

for train_index, test_index in tscv.split(unique_years):
    X_trimmed_train_ts = X_trimmed[X_trimmed.date.isin (unique_years[train_index])]
    X_trimmed_test_ts = X_trimmed[X_trimmed.date.isin (unique_years[test_index])]
    X_trimmed_train_ts.pop ('date')
    X_trimmed_test_ts.pop ('date')
    
    y_train_ts = y[y.date.isin (unique_years[train_index])]
    y_test_ts = y[y.date.isin (unique_years[test_index])]
    y_train_ts.pop ('date')
    y_test_ts.pop ('date')
    
    ts_scores_train.append(pipe_le.score (X_trimmed_train_ts, y_train_ts))
    ts_scores_test.append(pipe_le.score (X_trimmed_test_ts, y_test_ts))

print("\nKfold Cross Validation:")
print("Shuffled Cross-validated scores:", scores_shuffled)
print("Mean of Shuffled Cross-validated scores:", scores_shuffled.mean())

print("\nTime Series Cross Validation:")
print("Train Time-Series Cross Val mean score:", np.mean(ts_scores_train))
print("Test Time-Series Cross Val mean score:", np.mean(ts_scores_test))

print("\nTraining Score:", pipe_le.score(X_trimmed_train,y_train_le))
print("Dev Score:", pipe_le.score(X_trimmed_dev,y_dev_le))
print("Test Score:", pipe_le.score(X_trimmed_test,y_test_le))

print ("\nRoot of Mean Squared Error:", mean_squared_error (y_dev_le,pred_le)**0.5)

print ("\nNegative Values:", np.sum ((pred_le <= 0).values*1))

# Building a dataframe for coefficients
df_coef = pd.DataFrame(list(zip(X_trimmed_train.columns, pipe_le.named_steps['model'].coef_)))
df_coef.columns = ['predictors', 'coefficients']
df_coef['coef_abs'] = df_coef.coefficients.abs()
df_coef.sort_values (by='coef_abs', ascending = False, inplace=True)
df_coef[df_coef.coef_abs>0]


Kfold Cross Validation:
Shuffled Cross-validated scores: [0.88384733 0.88293602 0.90331678 0.87300036 0.86624841]
Mean of Shuffled Cross-validated scores: 0.8818697808162081

Time Series Cross Validation:
Train Time-Series Cross Val mean score: 0.8850720784762771
Test Time-Series Cross Val mean score: 0.8731404979127708

Training Score: 0.8841894402873267
Dev Score: 0.8499058380081029
Test Score: 0.8793603533590667

Root of Mean Squared Error: 3.1787258666237403

Negative Values: 0


,predictors,coefficients,coef_abs
15,Rural population with access to electricity (%),3.0837,3.0837
6,"Tuberculosis incidence (per 100,000 people)",-3.0068,3.0068
4,"Adolescent fertility rate (births per 1,000 wo...",-1.9492,1.9492
7,"Secondary education, general pupils (% female)",1.1460,1.1460
12,Mobile cellular subscriptions (per 100 people),0.9405,0.9405
14,"Government expenditure on education, total (% ...",-0.9018,0.9018
18,"Labor force participation rate, female (% of f...",-0.8517,0.8517
1,"School enrollment, primary (% gross)",0.6919,0.6919
11,Forest area (% of land area),0.6855,0.6855
16,"Unemployment, total (% of labour force)",-0.6064,0.6064


In [45]:
# Model for Mean years of schooling - LASSO
target = 'Mean years of schooling (years)'

y_train_mys = train_log[target]
y_dev_mys = dev_log[target]
y_test_mys = test_log[target]
y = work_log[['date'] + [target]]

model_mys = LassoCV(alphas=np.logspace(-4, 4, 100), cv=5, fit_intercept=True, max_iter=10000, tol = 0.01)

pipe_mys = Pipeline(steps=[
                       ('standardize', scaler_X),
                       ('model', model_mys)])

pipe_mys.fit (X_trimmed_train,y_train_mys)

pred_mys = pd.DataFrame(pipe_mys.predict (X_trimmed_dev))

#kfold cross validation
kf = KFold(n_splits=5, shuffle=True, random_state=1)
scores_shuffled = cross_val_score(pipe_mys, X_trimmed_train, y_train_mys, cv=kf)

#time_series cross validation
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

ts_scores_train = []
ts_scores_test = []

for train_index, test_index in tscv.split(unique_years):
    X_trimmed_train_ts = X_trimmed[X_trimmed.date.isin (unique_years[train_index])]
    X_trimmed_test_ts = X_trimmed[X_trimmed.date.isin (unique_years[test_index])]
    X_trimmed_train_ts.pop ('date')
    X_trimmed_test_ts.pop ('date')
    
    y_train_ts = y[y.date.isin (unique_years[train_index])]
    y_test_ts = y[y.date.isin (unique_years[test_index])]
    y_train_ts.pop ('date')
    y_test_ts.pop ('date')
    
    ts_scores_train.append(pipe_mys.score (X_trimmed_train_ts, y_train_ts))
    ts_scores_test.append(pipe_mys.score (X_trimmed_test_ts, y_test_ts))

print("\nKfold Cross Validation:")
print("Shuffled Cross-validated scores:", scores_shuffled)
print("Mean of Shuffled Cross-validated scores:", scores_shuffled.mean())

print("\nTime Series Cross Validation:")
print("Train Time-Series Cross Val mean score:", np.mean(ts_scores_train))
print("Test Time-Series Cross Val mean score:", np.mean(ts_scores_test))

print("\nTraining Score:", pipe_mys.score(X_trimmed_train,y_train_mys))
print("Dev Score:", pipe_mys.score(X_trimmed_dev,y_dev_mys))
print("Test Score:", pipe_mys.score(X_trimmed_test,y_test_mys))

print ("\nRoot of Mean Squared Error:", mean_squared_error (y_dev_mys,pred_mys)**0.5)

print ("\nNegative Values:", np.sum ((pred_mys <= 0).values*1))

# Building a dataframe for coefficients
df_coef = pd.DataFrame(list(zip(X_trimmed_train.columns, pipe_mys.named_steps['model'].coef_)))
df_coef.columns = ['predictors', 'coefficients']
df_coef['coef_abs'] = df_coef.coefficients.abs()
df_coef.sort_values (by='coef_abs', ascending = False, inplace=True)
df_coef[df_coef.coef_abs>0]


Kfold Cross Validation:
Shuffled Cross-validated scores: [0.86582461 0.84654388 0.86572127 0.8635942  0.85862915]
Mean of Shuffled Cross-validated scores: 0.8600626229981879

Time Series Cross Validation:
Train Time-Series Cross Val mean score: 0.8467645870487152
Test Time-Series Cross Val mean score: 0.8739593679126859

Training Score: 0.8632830124049016
Dev Score: 0.8769513469987974
Test Score: 0.8703920713732795

Root of Mean Squared Error: 1.1464622753546652

Negative Values: 0


,predictors,coefficients,coef_abs
0,Inequality in education (%),-1.4119,1.4119
3,"Agriculture, forestry, and fishing, value adde...",-0.5412,0.5412
15,Rural population with access to electricity (%),0.4186,0.4186
12,Mobile cellular subscriptions (per 100 people),0.3897,0.3897
5,"Pupil-teacher ratio, primary",-0.3463,0.3463
6,"Tuberculosis incidence (per 100,000 people)",0.3411,0.3411
8,"Out-of-pocket expenditure per capita, PPP (cur...",0.3176,0.3176
4,"Adolescent fertility rate (births per 1,000 wo...",-0.3116,0.3116
10,"PM2.5 pollution, population exposed to levels ...",0.2760,0.2760
13,International inbound tourists (thousands),-0.1785,0.1785


In [46]:
# Model for Expected years of schooling - LASSO
target = 'Expected years of schooling (years)'

y_train_eys = train_log[target]
y_dev_eys = dev_log[target]
y_test_eys = test_log[target]
y = work_log[['date'] + [target]]

model_eys = LassoCV(alphas=np.logspace(-4, 4, 100), cv=5, fit_intercept=True, max_iter=10000, tol = 0.01)

pipe_eys = Pipeline(steps=[
                       ('standardize', scaler_X),
                       ('model', model_eys)])

pipe_eys.fit (X_trimmed_train,y_train_eys)

pred_eys = pd.DataFrame(pipe_eys.predict (X_trimmed_dev))

#kfold cross validation
kf = KFold(n_splits=5, shuffle=True, random_state=1)
scores_shuffled = cross_val_score(pipe_eys, X_trimmed_train, y_train_eys, cv=kf)

#time_series cross validation
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

ts_scores_train = []
ts_scores_test = []

for train_index, test_index in tscv.split(unique_years):
    X_trimmed_train_ts = X_trimmed[X_trimmed.date.isin (unique_years[train_index])]
    X_trimmed_test_ts = X_trimmed[X_trimmed.date.isin (unique_years[test_index])]
    X_trimmed_train_ts.pop ('date')
    X_trimmed_test_ts.pop ('date')
    
    y_train_ts = y[y.date.isin (unique_years[train_index])]
    y_test_ts = y[y.date.isin (unique_years[test_index])]
    y_train_ts.pop ('date')
    y_test_ts.pop ('date')
    
    ts_scores_train.append(pipe_eys.score (X_trimmed_train_ts, y_train_ts))
    ts_scores_test.append(pipe_eys.score (X_trimmed_test_ts, y_test_ts))

print("\nKfold Cross Validation:")
print("Shuffled Cross-validated scores:", scores_shuffled)
print("Mean of Shuffled Cross-validated scores:", scores_shuffled.mean())

print("\nTime Series Cross Validation:")
print("Train Time-Series Cross Val mean score:", np.mean(ts_scores_train))
print("Test Time-Series Cross Val mean score:", np.mean(ts_scores_test))

print("\nTraining Score:", pipe_eys.score(X_trimmed_train,y_train_eys))
print("Dev Score:", pipe_eys.score(X_trimmed_dev,y_dev_eys))
print("Test Score:", pipe_eys.score(X_trimmed_test,y_test_eys))

print ("\nRoot of Mean Squared Error:", mean_squared_error (y_dev_eys,pred_eys)**0.5)

print ("\nNegative Values:", np.sum ((pred_eys <= 0).values*1))

# Building a dataframe for coefficients
df_coef = pd.DataFrame(list(zip(X_trimmed_train.columns, pipe_eys.named_steps['model'].coef_)))
df_coef.columns = ['predictors', 'coefficients']
df_coef['coef_abs'] = df_coef.coefficients.abs()
df_coef.sort_values (by='coef_abs', ascending = False, inplace=True)
df_coef[df_coef.coef_abs>0]


Kfold Cross Validation:
Shuffled Cross-validated scores: [0.87872838 0.87235383 0.88450669 0.88125859 0.86963124]
Mean of Shuffled Cross-validated scores: 0.8772957473500351

Time Series Cross Validation:
Train Time-Series Cross Val mean score: 0.8804844945429988
Test Time-Series Cross Val mean score: 0.8623014457155895

Training Score: 0.8792925045712294
Dev Score: 0.842002039218186
Test Score: 0.8663947742022005

Root of Mean Squared Error: 1.1405352484035398

Negative Values: 0


,predictors,coefficients,coef_abs
1,"School enrollment, primary (% gross)",0.9422,0.9422
2,Domestic general government health expenditure...,0.4552,0.4552
12,Mobile cellular subscriptions (per 100 people),0.4537,0.4537
5,"Pupil-teacher ratio, primary",-0.4429,0.4429
4,"Adolescent fertility rate (births per 1,000 wo...",-0.4233,0.4233
17,Corruption Perception Index,0.3350,0.3350
0,Inequality in education (%),-0.2925,0.2925
3,"Agriculture, forestry, and fishing, value adde...",-0.2851,0.2851
8,"Out-of-pocket expenditure per capita, PPP (cur...",0.2336,0.2336
10,"PM2.5 pollution, population exposed to levels ...",-0.2212,0.2212


In [47]:
# Testing final models on how they predict HDI versus actual HDI
gni_predictions = pipe_gni.predict (X_trimmed_test)
le_predictions = pipe_le.predict (X_trimmed_test)
mys_predictions = pipe_mys.predict (X_trimmed_test)
eys_predictions = pipe_eys.predict (X_trimmed_test)

hdi_predicted = []
for i in range(len(gni_predictions)):
    hdi = hdi_calculator (gni_predictions[i],le_predictions[i],mys_predictions[i],eys_predictions[i])
    hdi_predicted.append(hdi)
hdi_predicted = np.nan_to_num (hdi_predicted)

hdi_test = test_log['Human Development Index (HDI)']

# R2 score for predicted HDIs based on our models and actual HDIs
r2_score(hdi_test, hdi_predicted)

0.9752180596843791

In [48]:
# Testing final models on how they predict HDI versus actual HDI only for true values
test_clean = woiter[['country_code','country', 'date'] + predictors_again + target_indicators_all].dropna()
for col in logged_columns:
    test_clean[col] = np.log(test_clean[col])

print(test_clean.shape)
test_clean.head()

(405, 30)


,country_code,country,date,Inequality in education (%),"School enrollment, primary (% gross)","Domestic general government health expenditure per capita, PPP (current international $)","Agriculture, forestry, and fishing, value added (% of GDP)","Adolescent fertility rate (births per 1,000 women ages 15-19)","Pupil-teacher ratio, primary","Tuberculosis incidence (per 100,000 people)","Secondary education, general pupils (% female)","Out-of-pocket expenditure per capita, PPP (current international $)",Inequality in income (%),"PM2.5 pollution, population exposed to levels exceeding WHO Interim Target-3 value (% of total)",Forest area (% of land area),Mobile cellular subscriptions (per 100 people),International inbound tourists (thousands),"Government expenditure on education, total (% of GDP)",Rural population with access to electricity (%),"Unemployment, total (% of labour force)",Corruption Perception Index,"Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)",Mean years of schooling (years),Life expectancy index,Income index,Education index,Human Development Index (HDI),Gross national income (GNI) per capita (2011 PPP $),Life expectancy at birth (years),Expected years of schooling (years)
20,AGO,Angola,2010,26.2000,105.7810,4.5875,1.8214,172.9316,3.8196,5.9506,46.2789,3.4209,3.5946,100.0000,46.9078,40.2606,6.0521,1.2300,16.2100,2.2925,2.9444,75.3070,4.7264,0.5876,0.6031,0.3977,0.5204,5420.6528,58.1920,8.6462
51,ALB,Albania,2013,11.9000,99.9839,5.6704,2.9738,20.5124,2.9508,2.9444,48.8104,5.9050,2.9069,94.2903,28.2270,126.9370,7.9575,1.2639,100.0000,2.7473,3.4340,44.6500,9.6578,0.8877,0.7051,0.7323,0.7710,10648.3919,77.7020,14.7729
53,ALB,Albania,2015,11.9000,105.5430,5.7764,2.9847,20.0772,2.9240,2.7726,49.2511,6.0634,2.9086,93.2226,28.1569,117.6592,8.2385,1.2349,100.0000,2.8391,3.5835,47.6760,9.6798,0.8950,0.7112,0.7330,0.7756,11083.4795,78.1740,14.7729
54,ALB,Albania,2016,9.0840,105.5539,5.7499,2.9882,19.8596,2.9013,2.7726,49.5154,6.0878,2.9086,83.4325,28.1219,116.7444,8.3114,1.3749,100.0000,2.7213,3.6636,47.4590,9.9576,0.8976,0.7169,0.7423,0.7817,11512.3078,78.3450,14.7729
160,AUT,Austria,2010,2.4000,99.7898,8.0442,0.2369,9.5292,2.4005,2.2083,50.2718,6.6853,2.7147,52.0534,46.7476,145.5538,9.9990,1.7406,100.0000,1.5686,4.3694,53.5070,11.7838,0.9321,0.9184,0.8372,0.8949,43700.0727,80.5840,15.9985


In [49]:
X_test_clean = test_clean[predictors_again]
print (X_test_clean.shape)
X_test_clean.head()

(405, 19)


,Inequality in education (%),"School enrollment, primary (% gross)","Domestic general government health expenditure per capita, PPP (current international $)","Agriculture, forestry, and fishing, value added (% of GDP)","Adolescent fertility rate (births per 1,000 women ages 15-19)","Pupil-teacher ratio, primary","Tuberculosis incidence (per 100,000 people)","Secondary education, general pupils (% female)","Out-of-pocket expenditure per capita, PPP (current international $)",Inequality in income (%),"PM2.5 pollution, population exposed to levels exceeding WHO Interim Target-3 value (% of total)",Forest area (% of land area),Mobile cellular subscriptions (per 100 people),International inbound tourists (thousands),"Government expenditure on education, total (% of GDP)",Rural population with access to electricity (%),"Unemployment, total (% of labour force)",Corruption Perception Index,"Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)"
20,26.2000,105.7810,4.5875,1.8214,172.9316,3.8196,5.9506,46.2789,3.4209,3.5946,100.0000,46.9078,40.2606,6.0521,1.2300,16.2100,2.2925,2.9444,75.3070
51,11.9000,99.9839,5.6704,2.9738,20.5124,2.9508,2.9444,48.8104,5.9050,2.9069,94.2903,28.2270,126.9370,7.9575,1.2639,100.0000,2.7473,3.4340,44.6500
53,11.9000,105.5430,5.7764,2.9847,20.0772,2.9240,2.7726,49.2511,6.0634,2.9086,93.2226,28.1569,117.6592,8.2385,1.2349,100.0000,2.8391,3.5835,47.6760
54,9.0840,105.5539,5.7499,2.9882,19.8596,2.9013,2.7726,49.5154,6.0878,2.9086,83.4325,28.1219,116.7444,8.3114,1.3749,100.0000,2.7213,3.6636,47.4590
160,2.4000,99.7898,8.0442,0.2369,9.5292,2.4005,2.2083,50.2718,6.6853,2.7147,52.0534,46.7476,145.5538,9.9990,1.7406,100.0000,1.5686,4.3694,53.5070


In [50]:
# Testing final models on how they predict HDI versus actual HDI only for true values
gni_predictions = pipe_gni.predict (X_test_clean)
le_predictions = pipe_le.predict (X_test_clean)
mys_predictions = pipe_mys.predict (X_test_clean)
eys_predictions = pipe_eys.predict (X_test_clean)

hdi_predicted = []
for i in range(len(gni_predictions)):
    hdi = hdi_calculator (gni_predictions[i],le_predictions[i],mys_predictions[i],eys_predictions[i])
    hdi_predicted.append(hdi)
hdi_predicted = np.nan_to_num (hdi_predicted)

hdi_test = test_clean['Human Development Index (HDI)']

# R2 score for predicted HDIs based on our models and actual HDIs only for true values (excluding iterated values)
r2_score(hdi_test, hdi_predicted)

0.9748679812533938

### Step 3: Exporting model to Tableau through CSVs

In [56]:
# Now that the models are built, we need to export the data and the model coefficients into CSVs to take to Tableau

# Building a function to create coefficient tables for export
def coef_table_creator_lasso (fitted_model, predictor_df, name_str):
    coef_tbl = pd.DataFrame(list(zip((name_str+predictor_df.columns), fitted_model.coef_)), columns = ['Constant', 'Value'])
    intercept_tbl = pd.DataFrame({'Constant' : [name_str+'Intercept'], 
               'Value' : [fitted_model.intercept_]})
    final_table = pd.concat ([coef_tbl, intercept_tbl])
    return final_table


In [57]:
gni_tbl = coef_table_creator_lasso (pipe_gni.named_steps['model'], X_trimmed_train, 'GNI_')
le_tbl = coef_table_creator_lasso (pipe_le.named_steps['model'], X_trimmed_train, 'LE_')
mys_tbl = coef_table_creator_lasso (pipe_mys.named_steps['model'],  X_trimmed_train, 'MYS_')
eys_tbl = coef_table_creator_lasso (pipe_eys.named_steps['model'],  X_trimmed_train, 'EYS_')
std_tbl = pd.DataFrame(list(zip(('STD_'+ X_trimmed_train.columns), scaler_X.scale_)), columns = ['Constant', 'Value'])
mn_tbl = pd.DataFrame(list(zip(('MN_'+ X_trimmed_train.columns), scaler_X.mean_)), columns = ['Constant', 'Value'])

# Merging all the tables into one final table for export
model_table_one = pd.concat([gni_tbl, le_tbl,mys_tbl,eys_tbl, std_tbl, mn_tbl]).reset_index(drop=True)
model_table_one

,Constant,Value
0,GNI_Inequality in education (%),0.0000
1,"GNI_School enrollment, primary (% gross)",0.0153
2,GNI_Domestic general government health expendi...,0.4404
3,"GNI_Agriculture, forestry, and fishing, value ...",-0.2665
4,"GNI_Adolescent fertility rate (births per 1,00...",-0.0828
5,"GNI_Pupil-teacher ratio, primary",-0.0295
6,"GNI_Tuberculosis incidence (per 100,000 people)",-0.0000
7,"GNI_Secondary education, general pupils (% fem...",-0.0000
8,"GNI_Out-of-pocket expenditure per capita, PPP ...",0.1450
9,GNI_Inequality in income (%),-0.0017


In [58]:
# Exporting final model table for Tableau
model_table_one.index = model_table_one.Constant
model_table_one.pop('Constant')
model_table_one.T.to_csv ('../tableau_data/model_table_one.csv')

In [59]:
# Preparing clean (true values) data for Tableau export
data_clean = woiter[['country_code', 'country', 'date'] + predictors_again + target_indicators_all + ['High Inc Low Inc Class']]
print(data_clean.shape)
data_clean.head()

(4151, 31)


,country_code,country,date,Inequality in education (%),"School enrollment, primary (% gross)","Domestic general government health expenditure per capita, PPP (current international $)","Agriculture, forestry, and fishing, value added (% of GDP)","Adolescent fertility rate (births per 1,000 women ages 15-19)","Pupil-teacher ratio, primary","Tuberculosis incidence (per 100,000 people)","Secondary education, general pupils (% female)","Out-of-pocket expenditure per capita, PPP (current international $)",Inequality in income (%),"PM2.5 pollution, population exposed to levels exceeding WHO Interim Target-3 value (% of total)",Forest area (% of land area),Mobile cellular subscriptions (per 100 people),International inbound tourists (thousands),"Government expenditure on education, total (% of GDP)",Rural population with access to electricity (%),"Unemployment, total (% of labour force)",Corruption Perception Index,"Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)",Mean years of schooling (years),Life expectancy index,Income index,Education index,Human Development Index (HDI),Gross national income (GNI) per capita (2011 PPP $),Life expectancy at birth (years),Expected years of schooling (years),High Inc Low Inc Class
0,AGO,Angola,1990,nan,73.0396,nan,nan,216.6000,nan,nan,38.3306,nan,nan,100.0000,48.9099,nan,nan,nan,7.5186,nan,nan,74.9450,nan,0.3338,0.4563,nan,nan,2050.0510,41.6960,3.7809,nan
1,AGO,Angola,1991,nan,66.8044,nan,nan,214.8000,31.8767,nan,nan,nan,nan,nan,48.8098,nan,nan,nan,nan,23.7000,nan,74.8790,nan,0.3362,0.4529,nan,nan,2005.3859,41.8550,3.5662,nan
2,AGO,Angola,1992,nan,64.2471,nan,nan,213.0000,27.1810,nan,nan,nan,nan,nan,48.7097,nan,nan,nan,nan,nan,nan,74.8260,nan,0.3394,0.4498,nan,nan,1963.6370,42.0600,3.5471,nan
3,AGO,Angola,1993,nan,nan,nan,nan,210.9000,nan,nan,nan,nan,nan,nan,48.6096,0.0084,nan,nan,nan,nan,nan,74.7820,nan,0.3435,0.4028,nan,nan,1439.3732,42.3290,3.6570,nan
4,AGO,Angola,1994,nan,nan,nan,nan,208.8000,nan,nan,nan,nan,nan,nan,48.5095,0.0135,nan,nan,nan,nan,nan,74.7700,nan,0.3489,0.3625,nan,nan,1102.4029,42.6770,3.7668,nan


In [60]:
# Preparing iterated data for Tableau export
data_iterated = witer[['country_code', 'country', 'date'] + predictors_again + target_indicators_all + ['High Inc Low Inc Class']]
data_iterated['negative_vals'] = data_iterated[data_iterated[predictors_again + target_indicators_all] < 0].count(axis=1) >= 1
data_iterated = data_iterated[data_iterated.negative_vals == False]
data_iterated.pop('negative_vals')
print(data_iterated.shape)
data_iterated.head()

(3826, 31)


,country_code,country,date,Inequality in education (%),"School enrollment, primary (% gross)","Domestic general government health expenditure per capita, PPP (current international $)","Agriculture, forestry, and fishing, value added (% of GDP)","Adolescent fertility rate (births per 1,000 women ages 15-19)","Pupil-teacher ratio, primary","Tuberculosis incidence (per 100,000 people)","Secondary education, general pupils (% female)","Out-of-pocket expenditure per capita, PPP (current international $)",Inequality in income (%),"PM2.5 pollution, population exposed to levels exceeding WHO Interim Target-3 value (% of total)",Forest area (% of land area),Mobile cellular subscriptions (per 100 people),International inbound tourists (thousands),"Government expenditure on education, total (% of GDP)",Rural population with access to electricity (%),"Unemployment, total (% of labour force)",Corruption Perception Index,"Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)",Mean years of schooling (years),Life expectancy index,Income index,Education index,Human Development Index (HDI),Gross national income (GNI) per capita (2011 PPP $),Life expectancy at birth (years),Expected years of schooling (years),High Inc Low Inc Class
0,AGO,Angola,1990,31.7946,73.0396,47.1677,7.6161,216.6000,36.4299,357.3732,38.3306,55.7882,40.7507,100.0000,48.9099,3.0946,158.5374,2.5995,7.5186,22.6543,19.3559,74.9450,4.4740,0.3338,0.4563,0.1906,0.3119,2050.0510,41.6960,3.7809,nan
2,AGO,Angola,1992,31.4278,64.2471,53.9656,7.3658,213.0000,27.1810,327.4419,46.5205,53.5362,44.0560,100.0000,48.7097,0.7188,136.2831,2.5995,11.7030,23.3722,19.3559,74.8260,4.4386,0.3394,0.4498,0.2044,0.3195,1963.6370,42.0600,3.5471,nan
3,AGO,Angola,1993,31.7154,71.9530,50.7740,7.4091,210.9000,36.6296,322.2027,46.5205,52.8076,44.1965,100.0000,48.6096,0.0084,121.4084,2.5995,11.7030,23.7876,19.3559,74.7820,4.4819,0.3435,0.4028,0.2186,0.3267,1439.3732,42.3290,3.6570,nan
4,AGO,Angola,1994,31.3794,69.2585,54.2890,7.3115,208.8000,36.8101,288.0689,46.5205,51.2472,42.1421,100.0000,48.5095,0.0135,97.4764,2.5995,11.7030,24.2562,19.3559,74.7700,4.4571,0.3489,0.3625,0.2288,0.3310,1102.4029,42.6770,3.7668,nan
5,AGO,Angola,1995,31.1946,72.6726,62.9207,6.6532,206.7000,36.0369,330.1402,46.5205,50.5284,38.7731,100.0000,48.4094,0.0143,9.0000,2.5995,11.3216,23.9000,19.3559,74.7840,4.4203,0.3558,0.4432,0.2397,0.3424,1879.7079,43.1250,3.8767,nan


In [61]:
# Preparing logged data for Tableau export
data_log = work_log.copy()
print (data_log.shape)
data_log.head()

(2882, 30)


,country_code,country,date,Inequality in education (%),"School enrollment, primary (% gross)","Domestic general government health expenditure per capita, PPP (current international $)","Agriculture, forestry, and fishing, value added (% of GDP)","Adolescent fertility rate (births per 1,000 women ages 15-19)","Pupil-teacher ratio, primary","Tuberculosis incidence (per 100,000 people)","Secondary education, general pupils (% female)","Out-of-pocket expenditure per capita, PPP (current international $)",Inequality in income (%),"PM2.5 pollution, population exposed to levels exceeding WHO Interim Target-3 value (% of total)",Forest area (% of land area),Mobile cellular subscriptions (per 100 people),International inbound tourists (thousands),"Government expenditure on education, total (% of GDP)",Rural population with access to electricity (%),"Unemployment, total (% of labour force)",Corruption Perception Index,"Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate)",Mean years of schooling (years),Life expectancy index,Income index,Education index,Human Development Index (HDI),Gross national income (GNI) per capita (2011 PPP $),Life expectancy at birth (years),Expected years of schooling (years)
0,AGO,Angola,1990,31.7946,73.0396,3.8537,2.0303,216.6000,3.5954,5.8788,38.3306,4.0216,3.7075,100.0000,48.9099,3.0946,5.0660,0.9553,7.5186,3.1203,2.9630,74.9450,4.4740,0.3338,0.4563,0.1906,0.3119,2050.0510,41.6960,3.7809
2,AGO,Angola,1992,31.4278,64.2471,3.9883,1.9968,213.0000,3.3025,5.7913,46.5205,3.9804,3.7855,100.0000,48.7097,0.7188,4.9147,0.9553,11.7030,3.1515,2.9630,74.8260,4.4386,0.3394,0.4498,0.2044,0.3195,1963.6370,42.0600,3.5471
3,AGO,Angola,1993,31.7154,71.9530,3.9274,2.0027,210.9000,3.6009,5.7752,46.5205,3.9667,3.7886,100.0000,48.6096,0.0084,4.7992,0.9553,11.7030,3.1692,2.9630,74.7820,4.4819,0.3435,0.4028,0.2186,0.3267,1439.3732,42.3290,3.6570
4,AGO,Angola,1994,31.3794,69.2585,3.9943,1.9894,208.8000,3.6058,5.6632,46.5205,3.9367,3.7410,100.0000,48.5095,0.0135,4.5796,0.9553,11.7030,3.1887,2.9630,74.7700,4.4571,0.3489,0.3625,0.2288,0.3310,1102.4029,42.6770,3.7668
5,AGO,Angola,1995,31.1946,72.6726,4.1419,1.8951,206.7000,3.5845,5.7995,46.5205,3.9225,3.6577,100.0000,48.4094,0.0143,2.1972,0.9553,11.3216,3.1739,2.9630,74.7840,4.4203,0.3558,0.4432,0.2397,0.3424,1879.7079,43.1250,3.8767


In [62]:
# Exporting data for Tableau
data_clean.to_csv ('../tableau_data/data_clean.csv', index=False)
data_iterated.to_csv ('../tableau_data/data_iterated.csv', index=False)
data_log.to_csv ('../tableau_data/data_log.csv', index=False)

In [63]:
# Creating Model to generate feature importance for Tableau
target = 'Human Development Index (HDI)'

y_train_hdi_imp = train_log[target]
y_dev_hdi_imp = dev_log[target]
y_test_hdi_imp = test_log[target]
y = work_log[['date'] + [target]]

model_hdi_imp = LassoCV(alphas=np.logspace(-4, 4, 100), cv=5, fit_intercept=True, max_iter=10000, tol = 0.01)

pipe_hdi_imp = Pipeline(steps=[
                       ('standardize', scaler_X),
                       ('model', model_hdi_imp)])

pipe_hdi_imp.fit (X_trimmed_train,y_train_hdi_imp)

pred_hdi_imp = pd.DataFrame(pipe_hdi_imp.predict (X_trimmed_dev))

#kfold cross validation
kf = KFold(n_splits=5, shuffle=True, random_state=1)
scores_shuffled = cross_val_score(pipe_hdi_imp, X_trimmed_train, y_train_hdi_imp, cv=kf)

#time_series cross validation
from sklearn.model_selection import cross_val_score, TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

ts_scores_train = []
ts_scores_test = []

for train_index, test_index in tscv.split(unique_years):
    X_trimmed_train_ts = X_trimmed[X_trimmed.date.isin (unique_years[train_index])]
    X_trimmed_test_ts = X_trimmed[X_trimmed.date.isin (unique_years[test_index])]
    X_trimmed_train_ts.pop ('date')
    X_trimmed_test_ts.pop ('date')
    
    y_train_ts = y[y.date.isin (unique_years[train_index])]
    y_test_ts = y[y.date.isin (unique_years[test_index])]
    y_train_ts.pop ('date')
    y_test_ts.pop ('date')
    
    ts_scores_train.append(pipe_hdi_imp.score (X_trimmed_train_ts, y_train_ts))
    ts_scores_test.append(pipe_hdi_imp.score (X_trimmed_test_ts, y_test_ts))

print("\nKfold Cross Validation:")
print("Shuffled Cross-validated scores:", scores_shuffled)
print("Mean of Shuffled Cross-validated scores:", scores_shuffled.mean())

print("\nTime Series Cross Validation:")
print("Train Time-Series Cross Val mean score:", np.mean(ts_scores_train))
print("Test Time-Series Cross Val mean score:", np.mean(ts_scores_test))

print("\nTraining Score:", pipe_hdi_imp.score(X_trimmed_train,y_train_hdi_imp))
print("Dev Score:", pipe_hdi_imp.score(X_trimmed_dev,y_dev_hdi_imp))
print("Test Score:", pipe_hdi_imp.score(X_trimmed_test,y_test_hdi_imp))

print ("\nRoot of Mean Squared Error:", mean_squared_error (y_dev_hdi_imp,pred_hdi_imp)**0.5)

print ("\nNegative Values:", np.sum ((pred_hdi_imp <= 0).values*1))

# Building a dataframe for coefficients
df_coef = pd.DataFrame(list(zip(X_trimmed_train.columns, pipe_hdi_imp.named_steps['model'].coef_)))
df_coef.columns = ['predictors', 'coefficients']
df_coef['coef_abs'] = df_coef.coefficients.abs()
df_coef.sort_values (by='coef_abs', ascending = False, inplace=True)
df_coef[df_coef.coef_abs>0]


Kfold Cross Validation:
Shuffled Cross-validated scores: [0.96113767 0.95661151 0.96362586 0.96300767 0.95633503]
Mean of Shuffled Cross-validated scores: 0.9601435473765342

Time Series Cross Validation:
Train Time-Series Cross Val mean score: 0.9581707515112168
Test Time-Series Cross Val mean score: 0.96424562031977

Training Score: 0.9611161581809197
Dev Score: 0.9606729107492588
Test Score: 0.9722353136977218

Root of Mean Squared Error: 0.030729325711491787

Negative Values: 0


,predictors,coefficients,coef_abs
15,Rural population with access to electricity (%),0.0304,0.0304
2,Domestic general government health expenditure...,0.0269,0.0269
4,"Adolescent fertility rate (births per 1,000 wo...",-0.0218,0.0218
3,"Agriculture, forestry, and fishing, value adde...",-0.0215,0.0215
1,"School enrollment, primary (% gross)",0.0186,0.0186
0,Inequality in education (%),-0.0181,0.0181
8,"Out-of-pocket expenditure per capita, PPP (cur...",0.0153,0.0153
12,Mobile cellular subscriptions (per 100 people),0.0126,0.0126
17,Corruption Perception Index,0.0119,0.0119
5,"Pupil-teacher ratio, primary",-0.0108,0.0108


In [64]:
# Creating a relative_imp column to calculate relative importance to be used in Tableau
df_coef['relative_imp'] = df_coef.coef_abs.apply(lambda x: x/(np.sum(df_coef.coef_abs)))
df_coef

,predictors,coefficients,coef_abs,relative_imp
15,Rural population with access to electricity (%),0.0304,0.0304,0.1357
2,Domestic general government health expenditure...,0.0269,0.0269,0.1201
4,"Adolescent fertility rate (births per 1,000 wo...",-0.0218,0.0218,0.0972
3,"Agriculture, forestry, and fishing, value adde...",-0.0215,0.0215,0.0960
1,"School enrollment, primary (% gross)",0.0186,0.0186,0.0831
0,Inequality in education (%),-0.0181,0.0181,0.0809
8,"Out-of-pocket expenditure per capita, PPP (cur...",0.0153,0.0153,0.0681
12,Mobile cellular subscriptions (per 100 people),0.0126,0.0126,0.0564
17,Corruption Perception Index,0.0119,0.0119,0.0531
5,"Pupil-teacher ratio, primary",-0.0108,0.0108,0.0483


In [65]:
# Exporting feature importance file for Tableau usage
df_coef.to_csv ('../tableau_data/feature_importance.csv', index=False)

## Time for Tableau!

The final step of this project is to take the model and the data into Tableau to build an interactive dashboard that can actually be used by non-coding users to potentially make better hypothesis on what impacts poverty and how.